**Histórico dos dados dos fundos imobiliários**
* https://dados.cvm.gov.br/dataset/fii-doc-inf_mensal
* https://dados.cvm.gov.br/dataset/fii-doc-inf_trimestral
* https://dados.cvm.gov.br/dataset/fii-doc-inf_anual

---
O arquivo zipado 'inf_mensal_fii_XXXX' contém vários arquivos excel. Os arquivos que eu tenho interesse são:
* inf_mensal_fii_ativo_passivo_XXXX: contém os dados sobre "Obrigacoes_Aquisicao_Imoveis" e "Obrigacoes_Securitizacao_Recebiveis";
* inf_mensal_fii_complemento_XXXX: contém o dado sobre "Valor_Ativo".

In [1]:
from funcoes import *
import glob
import matplotlib.pyplot as plt
import os
import plotly.express as px
import zipfile

## Extraindo os arquivos zipados

### Extração dos arquivos "inf_mensal_fii_ativo_passivo_XXXX" e "inf_mensal_fii_complemento_XXXX"

In [ ]:
# Mês e ano escolhido
ano = '2025'

# Nome do arquivo zipado
zip_filename = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm_zip//inf_mensal_fii_{ano}.zip'

# # Visualizando a lista dos nomes dos arquivos dentro do arquivo zipado
# with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
#     print("Arquivos dentro do arquivo zipado:")
#     print(zip_ref.namelist())

# Lista dos nomes dos arquivos que você deseja extrair
arquivos_a_extrair = [
    f'inf_mensal_fii_ativo_passivo_{ano}.csv', 
    f'inf_mensal_fii_complemento_{ano}.csv', 
    f'inf_mensal_fii_geral_{ano}.csv'
]

# Caminho do diretório do diretório do arquivo não zipado
destino_dir = f'C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//{ano}'

# Abrindo o arquivo zipado
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    # Extraindo os arquivos selecionados para um diretório específico
    for arquivo in arquivos_a_extrair:
        zip_ref.extract(arquivo, destino_dir)

        # Caminhos dos arquivos em formato csv e parquet
        csv_path = f'{destino_dir}//{arquivo}'
        parquet_path = f'{destino_dir}//{arquivo.replace(".csv", ".parquet")}'

        # Lendo os arquivos csv
        df = pd.read_csv(csv_path, sep=';', encoding='ISO-8859-1', low_memory=False)

        # Transformando esses arquivos em parquet
        df.to_parquet(parquet_path)

# Entrando para o diretório especificado
os.chdir(destino_dir)

# Encontrando todos os arquivos .csv no diretório
arquivos_csv = glob.glob('*.csv')

# Loop para deletar cada arquivo csv encontrado
for arquivo in arquivos_csv:
    os.remove(arquivo)

## Fundos Imobiliários

In [2]:
# Lendo o arquivo da composição do IFIX
ifix_mai_ago_2024 = filtro_etf(path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//indices-b3//ifix//IFIX_2024_MAI_AGO.csv')
ifix_set_dez_2024 = filtro_etf(path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//indices-b3//ifix//IFIX_2024_SET_DEZ.csv')
ifix_jan_abr_2025 = filtro_etf(path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//indices-b3//ifix//IFIX_2025_JAN_ABR.csv')
ifix_mai_ago_2025 = filtro_etf(path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//indices-b3//ifix//IFIX_2025_MAI_AGO.csv')
ifix_set_dez_2025 = filtro_etf(path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//indices-b3//ifix//IFIX_2025_SET_DEZ.csv')

# Ordenando o df pela ordem alfabética dos tickers
ifix_mai_ago_2024 = ifix_mai_ago_2024.sort_values('Código')
ifix_set_dez_2024 = ifix_set_dez_2024.sort_values('Código')
ifix_jan_abr_2025 = ifix_jan_abr_2025.sort_values('Código')
ifix_mai_ago_2025 = ifix_mai_ago_2025.sort_values('Código')
ifix_set_dez_2025 = ifix_set_dez_2025.sort_values('Código')

# Transformando em listas
lista_ifix_atual = ifix_set_dez_2025['Código'].tolist()
lista_ifix_anterior = ifix_mai_ago_2025['Código'].tolist()

# Diferença entre as carteiras
fiis_sairam_carteira_atual = sorted(set(lista_ifix_anterior) - set(lista_ifix_atual))
fiis_entraram_carteira_atual = sorted(set(lista_ifix_atual) - set(lista_ifix_anterior))

print(f'A carteira teórioca do IFIX atual é composta por {len(lista_ifix_atual)} ações.')
print(f'A carteira teórioca do IFIX anterior é composta por {len(lista_ifix_anterior)} ações.')
print('-'*30)
print(f'O número de ações semelhantes são de {len(set(lista_ifix_atual) & set(lista_ifix_anterior))}.')
print(f'Ações semelhantes entre as duas carteiras: {sorted(set(lista_ifix_atual) & set(lista_ifix_anterior))}.')
print('-'*30)
print(f'Número de ações diferentes entre as duas carteiras: {len(set(lista_ifix_anterior) - set(lista_ifix_atual))}.')
print(f'FIIs que sairam da carteira atual: {fiis_sairam_carteira_atual}')
print(f'FIIs que entraram na carteira atual: {fiis_entraram_carteira_atual}')

A carteira teórioca do IFIX atual é composta por 115 ações.
A carteira teórioca do IFIX anterior é composta por 117 ações.
------------------------------
O número de ações semelhantes são de 112.
Ações semelhantes entre as duas carteiras: ['AFHI11', 'AIEC11', 'AJFI11', 'ALZR11', 'ARRI11', 'BBIG11', 'BCIA11', 'BCRI11', 'BLMG11', 'BPML11', 'BRCO11', 'BRCR11', 'BROF11', 'BTAL11', 'BTCI11', 'BTHF11', 'BTLG11', 'CACR11', 'CCME11', 'CLIN11', 'CPSH11', 'CPTS11', 'CVBI11', 'CYCR11', 'DEVA11', 'FATN11', 'GARE11', 'GGRC11', 'GTWR11', 'GZIT11', 'HABT11', 'HCTR11', 'HFOF11', 'HGBS11', 'HGCR11', 'HGLG11', 'HGRE11', 'HGRU11', 'HSAF11', 'HSLG11', 'HSML11', 'HTMX11', 'ICRI11', 'IRDM11', 'ITRI11', 'JSAF11', 'JSRE11', 'KCRE11', 'KFOF11', 'KISU11', 'KIVO11', 'KNCR11', 'KNHF11', 'KNHY11', 'KNIP11', 'KNRI11', 'KNSC11', 'KNUQ11', 'KORE11', 'LIFE11', 'LVBI11', 'MANA11', 'MCCI11', 'MCRE11', 'MFII11', 'MXRF11', 'OUJP11', 'PATL11', 'PMIS11', 'PORD11', 'PVBI11', 'RBFF11', 'RBRF11', 'RBRL11', 'RBRP11', 'RBRR11', 

* Esses CNPJs do "dict_cnpj" foram adicionados na mão. Eu não achei uma maneira de automatizar esse processo.

In [3]:
# IFIX 2024 (MAI-AGO)
dict_cnpj_mai_ago_2024 = {
    'KNIP11':'24.960.430/0001-13',
    'XPML11':'28.757.546/0001-00',
    'KNCR11':'16.706.958/0001-32',
    'HGLG11':'11.728.688/0001-47',
    'KNRI11':'12.005.956/0001-65',
    'BTLG11':'11.839.593/0001-09',
    'VISC11':'17.554.274/0001-25',
    'MXRF11':'97.521.225/0001-25',
    'XPLG11':'26.502.794/0001-85',
    'KNHY11':'30.130.708/0001-28',
    'HGBS11':'08.431.747/0001-06',
    'IRDM11':'28.830.325/0001-10',
    'CPTS11':'18.979.895/0001-13',
    'PVBI11':'35.652.102/0001-76',
    'HGRU11':'29.641.226/0001-53',
    'TGAR11':'25.032.881/0001-53',
    'RECR11':'28.152.272/0001-26',
    'BRCO11':'20.748.515/0001-81',
    'HSML11':'32.892.018/0001-31',
    'KNHF11':'42.754.342/0001-47',
    'TRXF11':'28.548.288/0001-52',
    'LVBI11':'30.629.603/0001-18',
    'BCFF11':'11.026.627/0001-38',
    'RZTR11':'36.501.128/0001-86',
    'HFOF11':'18.307.582/0001-19',
    'TVRI11':'14.410.722/0001-29',
    'HGCR11':'11.160.521/0001-22',
    'MCCI11':'23.648.935/0001-84',
    'VGHF11':'36.771.692/0001-19',
    'MALL11':'26.499.833/0001-32',
    'BRCR11':'08.924.783/0001-01',
    'HGRE11':'09.072.017/0001-29',
    'JSRE11':'13.371.132/0001-71',
    'RBRY11':'30.166.700/0001-11',
    'VILG11':'24.853.044/0001-22',
    'RBRR11':'29.467.977/0001-03',
    'RBVA11':'15.576.907/0001-70',
    'VCJR11':'32.400.250/0001-05',
    'VRTA11':'11.664.201/0001-00',
    'KNSC11':'35.864.448/0001-38',
    'ALZR11':'28.737.771/0001-85',
    'GARE11':'37.295.919/0001-60',
    'GGRC11':'26.614.291/0001-00',
    'HSLG11':'32.903.621/0001-71',
    'RBRF11':'27.529.279/0001-51',
    'VGIP11':'34.197.811/0001-46',
    'MCHY11':'36.655.973/0001-06',
    'CVBI11':'28.729.197/0001-13',
    'KORE11':'52.219.978/0001-42',
    'BTCI11':'09.552.812/0001-14',
    'GTWR11':'23.740.527/0001-58',
    'VGIR11':'29.852.732/0001-91',
    'URPR11':'34.508.872/0001-87',
    'RVBI11':'35.507.457/0001-71',
    'TRBL11':'16.671.412/0001-93',
    'RZAK11':'36.642.219/0001-31',
    'XPCI11':'28.516.301/0001-91',
    'JSAF11':'42.085.661/0001-07',
    'RBRP11':'21.408.063/0001-51',
    'HABT11':'30.578.417/0001-05',
    'HCTR11':'30.248.180/0001-96',
    'DEVA11':'37.087.810/0001-37',
    'VINO11':'12.516.185/0001-70',
    'BROF11':'48.978.859/0001-04',
    'RCRB11':'03.683.056/0001-86',
    'XPIN11':'28.516.325/0001-40',
    'RBRL11':'35.705.463/0001-33',
    'HGPO11':'11.260.134/0001-68',
    'MFII11':'16.915.968/0001-88',
    'BCRI11':'22.219.335/0001-38',
    'SARE11':'32.903.702/0001-71',
    'KFOF11':'30.091.444/0001-40',
    'CLIN11':'49.005.348/0001-60',
    'BTAL11':'36.642.244/0001-15',
    'TEPP11':'26.681.370/0001-25',
    'SNCI11':'41.076.710/0001-82',
    'CACR11':'32.065.364/0001-46',
    'AFHI11':'36.642.293/0001-58',
    'BCIA11':'20.216.935/0001-17',
    'BARI11':'29.267.567/0001-00',
    'KISU11':'36.669.660/0001-07',
    'RZAT11':'28.267.696/0001-36',
    'XPSF11':'30.983.020/0001-90',
    'KCRE11':'42.502.802/0001-40',
    'PORD11':'17.156.502/0001-09',
    'RECT11':'32.274.163/0001-59',
    'BPFF11':'17.324.357/0001-28',
    'PATL11':'35.754.164/0001-99',
    'LGCP11':'34.598.181/0001-11',
    'OUJP11':'26.091.656/0001-50',
    'AJFI11':'51.472.985/0001-99',
    'RBRX11':'41.088.458/0001-21',
    'HTMX11':'08.706.065/0001-69',
    'AIEC11':'35.765.826/0001-26',
    'HGFF11':'32.784.898/0001-22',
    'SNFF11':'40.011.225/0001-68',
    'WHGR11':'41.256.643/0001-88',
    'RBFF11':'17.329.029/0001-14',
    'FATN11':'30.567.216/0001-02',
    'HSAF11':'35.360.687/0001-50',
    'CPFF11':'34.081.611/0001-23',
    'SPXS11':'43.010.543/0001-00',
    'VIUR11':'36.445.587/0001-90',
    'PLCR11':'32.527.683/0001-26',
    'BTRA11':'41.076.607/0001-32',
    'BLMG11':'34.081.637/0001-71',
    'ARRI11':'32.006.821/0001-21',
    'CYCR11':'36.501.233/0001-15',
    'XPPR11':'30.654.849/0001-40',
    'NCHB11':'18.085.673/0001-57',
    'VSLH11':'36.244.015/0001-42',
    'TORD11':'30.230.870/0001-18'
}

# IFIX 2024 (SET-DEZ)
dict_cnpj_set_dez_2024 = {
    'KNIP11':'24.960.430/0001-13',
    'KNCR11':'16.706.958/0001-32',
    'XPML11':'28.757.546/0001-00',
    'HGLG11':'11.728.688/0001-47',
    'BTLG11':'11.839.593/0001-09',
    'MXRF11':'97.521.225/0001-25',
    'KNRI11':'12.005.956/0001-65',
    'HGRU11':'29.641.226/0001-53',
    'VISC11':'17.554.274/0001-25',
    'XPLG11':'26.502.794/0001-85',
    'KNHY11':'30.130.708/0001-28',
    'TGAR11':'25.032.881/0001-53',
    'HGBS11':'08.431.747/0001-06',
    'IRDM11':'28.830.325/0001-10',
    'CPTS11':'18.979.895/0001-13',
    'PVBI11':'35.652.102/0001-76',
    'RECR11':'28.152.272/0001-26',
    'TRXF11':'28.548.288/0001-52',
    'HSML11':'32.892.018/0001-31',
    'KNHF11':'42.754.342/0001-47',
    'BRCO11':'20.748.515/0001-81',
    'LVBI11':'30.629.603/0001-18',
    'RZTR11':'36.501.128/0001-86',
    'KNSC11':'35.864.448/0001-38',
    'BCFF11':'11.026.627/0001-38',
    'TVRI11':'14.410.722/0001-29',
    'KNUQ11':'42.754.362/0001-18',
    'HFOF11':'18.307.582/0001-19',
    'HGCR11':'11.160.521/0001-22',
    'VGIR11':'29.852.732/0001-91',
    'MCCI11':'23.648.935/0001-84',
    'VGHF11':'36.771.692/0001-19',
    'MALL11':'26.499.833/0001-32',
    'VRTA11':'11.664.201/0001-00',
    'RBRR11':'29.467.977/0001-03',
    'VCJR11':'32.400.250/0001-05',
    'BRCR11':'08.924.783/0001-01',
    'HGRE11':'09.072.017/0001-29',
    'RBVA11':'15.576.907/0001-70',
    'JSRE11':'13.371.132/0001-71',
    'ALZR11':'28.737.771/0001-85',
    'VILG11':'24.853.044/0001-22',
    'GZIT11':'15.447.108/0001-02',
    'RBRY11':'30.166.700/0001-11',
    'GGRC11':'26.614.291/0001-00',
    'GARE11':'37.295.919/0001-60',
    'HSLG11':'32.903.621/0001-71',
    'MCHY11':'36.655.973/0001-06',
    'VGIP11':'34.197.811/0001-46',
    'CVBI11':'28.729.197/0001-13',
    'RBRF11':'27.529.279/0001-51',
    'KORE11':'52.219.978/0001-42',
    'BTCI11':'09.552.812/0001-14',
    'GTWR11':'23.740.527/0001-58',
    'URPR11':'34.508.872/0001-87',
    'JSAF11':'42.085.661/0001-07',
    'RVBI11':'35.507.457/0001-71',
    'XPCI11':'28.516.301/0001-91',
    'TRBL11':'16.671.412/0001-93',
    'RZAK11':'36.642.219/0001-31',
    'HABT11':'30.578.417/0001-05',
    'RBRP11':'21.408.063/0001-51',
    'CPSH11':'47.896.665/0001-99',
    'BPML11':'33.046.142/0001-49',
    'HCTR11':'30.248.180/0001-96',
    'KFOF11':'30.091.444/0001-40',
    'DEVA11':'37.087.810/0001-37',
    'XPIN11':'28.516.325/0001-40',
    'BROF11':'48.978.859/0001-04',
    'MFII11':'16.915.968/0001-88',
    'RBRL11':'35.705.463/0001-33',
    'HGPO11':'11.260.134/0001-68',
    'CACR11':'32.065.364/0001-46',
    'VINO11':'12.516.185/0001-70',
    'RCRB11':'03.683.056/0001-86',
    'BTAL11':'36.642.244/0001-15',
    'AFHI11':'36.642.293/0001-58',
    'BCRI11':'22.219.335/0001-38',
    'CLIN11':'49.005.348/0001-60',
    'SNCI11':'41.076.710/0001-82',
    'BCIA11':'20.216.935/0001-17',
    'TEPP11':'26.681.370/0001-25',
    'SARE11':'32.903.702/0001-71',
    'BARI11':'29.267.567/0001-00',
    'RZAT11':'28.267.696/0001-36',
    'KISU11':'36.669.660/0001-07',
    'KCRE11':'42.502.802/0001-40',
    'MANA11':'42.888.583/0001-89',
    'PORD11':'17.156.502/0001-09',
    'XPSF11':'30.983.020/0001-90',
    'FATN11':'30.567.216/0001-02',
    'BPFF11':'17.324.357/0001-28',
    'PATL11':'35.754.164/0001-99',
    'WHGR11':'41.256.643/0001-88',
    'LGCP11':'34.598.181/0001-11',
    'RECT11':'32.274.163/0001-59',
    'HTMX11':'08.706.065/0001-69',
    'RBRX11':'41.088.458/0001-21',
    'AIEC11':'35.765.826/0001-26',
    'AJFI11':'51.472.985/0001-99',
    'OUJP11':'26.091.656/0001-50',
    'SNFF11':'40.011.225/0001-68',
    'HGFF11':'32.784.898/0001-22',
    'RBFF11':'17.329.029/0001-14',
    'HSAF11':'35.360.687/0001-50',
    'SPXS11':'43.010.543/0001-00',
    'BLMG11':'34.081.637/0001-71',
    'KIVO11':'42.273.325/0001-98',
    'VIUR11':'36.445.587/0001-90',
    'SNEL11':'43.741.171/0001-84',
    'ARRI11':'32.006.821/0001-21',
    'CYCR11':'36.501.233/0001-15',
    'BTRA11':'41.076.607/0001-32',
    'ZAVI11':'40.575.940/0001-23',
    'VSLH11':'36.244.015/0001-42'
}

# IFIX 2025 (JAN-ABR)
dict_cnpj_jan_abr_2025 = {
    'KNCR11':'16.706.958/0001-32',
    'KNIP11':'24.960.430/0001-13',
    'XPML11':'28.757.546/0001-00',
    'HGLG11':'11.728.688/0001-47',
    'BTLG11':'11.839.593/0001-09',
    'MXRF11':'97.521.225/0001-25',
    'KNRI11':'12.005.956/0001-65',
    'XPLG11':'26.502.794/0001-85',
    'KNHY11':'30.130.708/0001-28',
    'VISC11':'17.554.274/0001-25',
    'HGRU11':'29.641.226/0001-53',
    'HGBS11':'08.431.747/0001-06',
    'IRDM11':'28.830.325/0001-10',
    'CPTS11':'18.979.895/0001-13',
    'PVBI11':'35.652.102/0001-76',
    'TGAR11':'25.032.881/0001-53',
    'TRXF11':'28.548.288/0001-52',
    'RECR11':'28.152.272/0001-26',
    'KNSC11':'35.864.448/0001-38',
    'KNHF11':'42.754.342/0001-47',
    'RZTR11':'36.501.128/0001-86',
    'BRCO11':'20.748.515/0001-81',
    'KNUQ11':'42.754.362/0001-18',
    'LVBI11':'30.629.603/0001-18',
    'GARE11':'37.295.919/0001-60',
    'HSML11':'32.892.018/0001-31',
    'HGCR11':'11.160.521/0001-22',
    'TVRI11':'14.410.722/0001-29',
    'RBVA11':'15.576.907/0001-70',
    'VGIR11':'29.852.732/0001-91',
    'MCCI11':'23.648.935/0001-84',
    'HFOF11':'18.307.582/0001-19',
    'MALL11':'26.499.833/0001-32',
    'RBRR11':'29.467.977/0001-03',
    'VGHF11':'36.771.692/0001-19',
    'ALZR11':'28.737.771/0001-85',
    'VRTA11':'11.664.201/0001-00',
    'HGRE11':'09.072.017/0001-29',
    'GGRC11':'26.614.291/0001-00',
    'JSRE11':'13.371.132/0001-71',
    'RBRY11':'30.166.700/0001-11',
    'VCJR11':'32.400.250/0001-05',
    'VILG11':'24.853.044/0001-22',
    'BRCR11':'08.924.783/0001-01',
    'HSLG11':'32.903.621/0001-71',
    'VGIP11':'34.197.811/0001-46',
    'GZIT11':'15.447.108/0001-02',
    'CVBI11':'28.729.197/0001-13',
    'RBRF11':'27.529.279/0001-51',
    'BTCI11':'09.552.812/0001-14',
    'MCRE11':'36.655.973/0001-06',
    'GTWR11':'23.740.527/0001-58',
    'BBIG11':'54.375.187/0001-37',
    'KORE11':'52.219.978/0001-42',
    'CPSH11':'47.896.665/0001-99',
    'RZAK11':'36.642.219/0001-31',
    'URPR11':'34.508.872/0001-87',
    'XPCI11':'28.516.301/0001-91',
    'RVBI11':'35.507.457/0001-71',
    'HABT11':'30.578.417/0001-05',
    'RBRP11':'21.408.063/0001-51',
    'SNEL11':'43.741.171/0001-84',
    'CCME11':'43.010.844/0001-26',
    'JSAF11':'42.085.661/0001-07',
    'BPML11':'33.046.142/0001-49',
    'TRBL11':'16.671.412/0001-93',
    'BROF11':'48.978.859/0001-04',
    'HTMX11':'08.706.065/0001-69',
    'XPIN11':'28.516.325/0001-40',
    'KFOF11':'30.091.444/0001-40',
    'RCRB11':'03.683.056/0001-86',
    'MFII11':'16.915.968/0001-88',
    'RBRL11':'35.705.463/0001-33',
    'ITRI11':'52.270.671/0001-76',
    'DEVA11':'37.087.810/0001-37',
    'HCTR11':'30.248.180/0001-96',
    'AFHI11':'36.642.293/0001-58',
    'BTAL11':'36.642.244/0001-15',
    'CACR11':'32.065.364/0001-46',
    'VINO11':'12.516.185/0001-70',
    'SNCI11':'41.076.710/0001-82',
    'CLIN11':'49.005.348/0001-60',
    'RZAT11':'28.267.696/0001-36',
    'BCRI11':'22.219.335/0001-38',
    'SNFF11':'40.011.225/0001-68',
    'SARE11':'32.903.702/0001-71',
    'TEPP11':'26.681.370/0001-25',
    'ICRI11':'51.294.441/0001-84',
    'BARI11':'29.267.567/0001-00',
    'KCRE11':'42.502.802/0001-40',
    'BCIA11':'20.216.935/0001-17',
    'MANA11':'42.888.583/0001-89',
    'KISU11':'36.669.660/0001-07',
    'CYCR11':'36.501.233/0001-15',
    'LIFE11':'39.753.295/0001-02',
    'PORD11':'17.156.502/0001-09',
    'FATN11':'30.567.216/0001-02',
    'XPSF11':'30.983.020/0001-90',
    'VGRI11':'53.656.482/0001-07',
    'WHGR11':'41.256.643/0001-88',
    'BPFF11':'17.324.357/0001-28',
    'RECT11':'32.274.163/0001-59',
    'OUJP11':'26.091.656/0001-50',
    'PATL11':'35.754.164/0001-99',
    'RBRX11':'41.088.458/0001-21',
    'AJFI11':'51.472.985/0001-99',
    'HGPO11':'11.260.134/0001-68',
    'AIEC11':'35.765.826/0001-26',
    'HGFF11':'32.784.898/0001-22',
    'HSAF11':'35.360.687/0001-50',
    'RBFF11':'17.329.029/0001-14',
    'SPXS11':'43.010.543/0001-00',
    'VIUR11':'36.445.587/0001-90',
    'ARRI11':'32.006.821/0001-21',
    'BTRA11':'41.076.607/0001-32',
    'KIVO11':'42.273.325/0001-98',
    'BLMG11':'34.081.637/0001-71'
}

# IFIX 2025 (MAI-AGO)
dict_cnpj_mai_ago_2025 = {
    'KNCR11':'16.706.958/0001-32',
    'KNIP11':'24.960.430/0001-13',
    'XPML11':'28.757.546/0001-00',
    'HGLG11':'11.728.688/0001-47',
    'BTLG11':'11.839.593/0001-09',
    'KNRI11':'12.005.956/0001-65',
    'MXRF11':'97.521.225/0001-25',
    'XPLG11':'26.502.794/0001-85',
    'VISC11':'17.554.274/0001-25',
    'HGRU11':'29.641.226/0001-53',
    'KNHY11':'30.130.708/0001-28',
    'IRDM11':'28.830.325/0001-10',
    'HGBS11':'08.431.747/0001-06',
    'CPTS11':'18.979.895/0001-13',
    'RECR11':'28.152.272/0001-26',
    'PVBI11':'35.652.102/0001-76',
    'TRXF11':'28.548.288/0001-52',
    'TGAR11':'25.032.881/0001-53',
    'KNHF11':'42.754.342/0001-47',
    'HSML11':'32.892.018/0001-31',
    'BRCO11':'20.748.515/0001-81',
    'KNSC11':'35.864.448/0001-38',
    'BTHF11':'45.188.176/0001-57',
    'RZTR11':'36.501.128/0001-86',
    'LVBI11':'30.629.603/0001-18',
    'KNUQ11':'42.754.362/0001-18',
    'HGCR11':'11.160.521/0001-22',
    'MALL11':'26.499.833/0001-32',
    'MCCI11':'23.648.935/0001-84',
    'TVRI11':'14.410.722/0001-29',
    'HFOF11':'18.307.582/0001-19',
    'VGIR11':'29.852.732/0001-91',
    'HGRE11':'09.072.017/0001-29',
    'RBVA11':'15.576.907/0001-70',
    'GARE11':'37.295.919/0001-60',
    'JSRE11':'13.371.132/0001-71',
    'VILG11':'24.853.044/0001-22',
    'RBRR11':'29.467.977/0001-03',
    'VRTA11':'11.664.201/0001-00',
    'VGHF11':'36.771.692/0001-19',
    'VCJR11':'32.400.250/0001-05',
    'GGRC11':'26.614.291/0001-00',
    'ALZR11':'28.737.771/0001-85',
    'RBRY11':'30.166.700/0001-11',
    'BRCR11':'08.924.783/0001-01',
    'GZIT11':'15.447.108/0001-02',
    'HSLG11':'32.903.621/0001-71',
    'MCRE11':'36.655.973/0001-06',
    'VGIP11':'34.197.811/0001-46',
    'RBRF11':'27.529.279/0001-51',
    'CVBI11':'28.729.197/0001-13',
    'BTCI11':'09.552.812/0001-14',
    'GTWR11':'23.740.527/0001-58',
    'KORE11':'52.219.978/0001-42',
    'CPSH11':'47.896.665/0001-99',
    'BBIG11':'54.375.187/0001-37',
    'RZAK11':'36.642.219/0001-31',
    'RVBI11':'35.507.457/0001-71',
    'XPCI11':'28.516.301/0001-91',
    'HABT11':'30.578.417/0001-05',
    'CCME11':'43.010.844/0001-26',
    'JSAF11':'42.085.661/0001-07',
    'URPR11':'34.508.872/0001-87',
    'RBRP11':'21.408.063/0001-51',
    'BPML11':'33.046.142/0001-49',
    'BROF11':'48.978.859/0001-04',
    'KFOF11':'30.091.444/0001-40',
    'RBRL11':'35.705.463/0001-33',
    'MFII11':'16.915.968/0001-88',
    'ITRI11':'52.270.671/0001-76',
    'RCRB11':'03.683.056/0001-86',
    'TRBL11':'16.671.412/0001-93',
    'HCTR11':'30.248.180/0001-96',
    'CACR11':'32.065.364/0001-46',
    'BTAL11':'36.642.244/0001-15',
    'AFHI11':'36.642.293/0001-58',
    'SARE11':'32.903.702/0001-71',
    'VINO11':'12.516.185/0001-70',
    'DEVA11':'37.087.810/0001-37',
    'BCRI11':'22.219.335/0001-38',
    'HTMX11':'08.706.065/0001-69',
    'CLIN11':'49.005.348/0001-60',
    'SNCI11':'41.076.710/0001-82',
    'RZAT11':'28.267.696/0001-36',
    'TEPP11':'26.681.370/0001-25',
    'ICRI11':'51.294.441/0001-84',
    'BARI11':'29.267.567/0001-00',
    'TOPP11':'56.805.391/0001-76',
    'SNEL11':'43.741.171/0001-84',
    'VRTM11':'51.870.412/0001-13',
    'FATN11':'30.567.216/0001-02',
    'LIFE11':'39.753.295/0001-02',
    'MANA11':'42.888.583/0001-89',
    'BCIA11':'20.216.935/0001-17',
    'CYCR11':'36.501.233/0001-15',
    'KISU11':'36.669.660/0001-07',
    'KCRE11':'42.502.802/0001-40',
    'PORD11':'17.156.502/0001-09',
    'SNFF11':'40.011.225/0001-68',
    'VGRI11':'53.656.482/0001-07',
    'XPSF11':'30.983.020/0001-90',
    'WHGR11':'41.256.643/0001-88',
    'OUJP11':'26.091.656/0001-50',
    'PATL11':'35.754.164/0001-99',
    'RBRX11':'41.088.458/0001-21',
    'AJFI11':'51.472.985/0001-99',
    'HGPO11':'11.260.134/0001-68',
    'AIEC11':'35.765.826/0001-26',
    'HGFF11':'32.784.898/0001-22',
    'HSAF11':'35.360.687/0001-50',
    'RBFF11':'17.329.029/0001-14',
    'SPXS11':'43.010.543/0001-00',
    'ARRI11':'32.006.821/0001-21',
    'VIUR11':'36.445.587/0001-90',
    'KIVO11':'42.273.325/0001-98',
    'BLMG11':'34.081.637/0001-71',
    'PMIS11':'51.868.778/0001-58'
}

# IFIX 2025 (SET-DEZ)
dict_cnpj_set_dez_2025 = {
    'AFHI11':'36.642.293/0001-58',
    'AIEC11':'35.765.826/0001-26',
    'AJFI11':'51.472.985/0001-99',
    'ALZR11':'28.737.771/0001-85',
    'ARRI11':'32.006.821/0001-21',
    'BBIG11':'54.375.187/0001-37',
    'BCIA11':'20.216.935/0001-17',
    'BCRI11':'22.219.335/0001-38',
    'BLMG11':'34.081.637/0001-71',
    'BPML11':'33.046.142/0001-49',
    'BRCO11':'20.748.515/0001-81',
    'BRCR11':'08.924.783/0001-01',
    'BROF11':'48.978.859/0001-04',
    'BTAL11':'36.642.244/0001-15',
    'BTCI11':'09.552.812/0001-14',
    'BTHF11':'45.188.176/0001-57',
    'BTLG11':'11.839.593/0001-09',
    'CACR11':'32.065.364/0001-46',
    'CCME11':'43.010.844/0001-26',
    'CLIN11':'49.005.348/0001-60',
    'CPSH11':'47.896.665/0001-99',
    'CPTS11':'18.979.895/0001-13',
    'CVBI11':'28.729.197/0001-13',
    'CYCR11':'36.501.233/0001-15',
    'DEVA11':'37.087.810/0001-37',
    'FATN11':'30.567.216/0001-02',
    'GARE11':'37.295.919/0001-60',
    'GGRC11':'26.614.291/0001-00',
    'GTWR11':'23.740.527/0001-58',
    'GZIT11':'15.447.108/0001-02',
    'HABT11':'30.578.417/0001-05',
    'HCTR11':'30.248.180/0001-96',
    'HFOF11':'18.307.582/0001-19',
    'HGBS11':'08.431.747/0001-06',
    'HGCR11':'11.160.521/0001-22',
    'HGLG11':'11.728.688/0001-47',
    'HGRE11':'09.072.017/0001-29',
    'HGRU11':'29.641.226/0001-53',
    'HSAF11':'35.360.687/0001-50',
    'HSLG11':'32.903.621/0001-71',
    'HSML11':'32.892.018/0001-31',
    'HTMX11':'08.706.065/0001-69',
    'ICRI11':'51.294.441/0001-84',
    'IRDM11':'28.830.325/0001-10',
    'ITRI11':'52.270.671/0001-76',
    'JSAF11':'42.085.661/0001-07',
    'JSCR11':'54.866.214/0001-74',
    'JSRE11':'13.371.132/0001-71',
    'KCRE11':'42.502.802/0001-40',
    'KFOF11':'30.091.444/0001-40',
    'KISU11':'36.669.660/0001-07',
    'KIVO11':'42.273.325/0001-98',
    'KNCR11':'16.706.958/0001-32',
    'KNHF11':'42.754.342/0001-47',
    'KNHY11':'30.130.708/0001-28',
    'KNIP11':'24.960.430/0001-13',
    'KNRI11':'12.005.956/0001-65',
    'KNSC11':'35.864.448/0001-38',
    'KNUQ11':'42.754.362/0001-18',
    'KORE11':'52.219.978/0001-42',
    'LIFE11':'39.753.295/0001-02',   
    'LVBI11':'30.629.603/0001-18',
    'MANA11':'42.888.583/0001-89',
    'MCCI11':'23.648.935/0001-84',
    'MCRE11':'36.655.973/0001-06',
    'MFII11':'16.915.968/0001-88',
    'MXRF11':'97.521.225/0001-25',
    'OUJP11':'26.091.656/0001-50',
    'PATL11':'35.754.164/0001-99',
    'PMIS11':'51.868.778/0001-58',
    'PMLL11':'26.499.833/0001-32',
    'PORD11':'17.156.502/0001-09',
    'PVBI11':'35.652.102/0001-76',
    'RBFF11':'17.329.029/0001-14',
    'RBVA11':'15.576.907/0001-70',
    'RBRF11':'27.529.279/0001-51',
    'RBRL11':'35.705.463/0001-33',
    'RBRP11':'21.408.063/0001-51',
    'RBRR11':'29.467.977/0001-03',
    'RBRX11':'41.088.458/0001-21',
    'RBRY11':'30.166.700/0001-11',
    'RBVA11':'15.576.907/0001-70',
    'RCRB11':'03.683.056/0001-86',
    'RECR11':'28.152.272/0001-26',
    'RVBI11':'35.507.457/0001-71',
    'RZAK11':'36.642.219/0001-31',
    'RZAT11':'28.267.696/0001-36',
    'RZTR11':'36.501.128/0001-86',
    'SARE11':'32.903.702/0001-71',
    'SNCI11':'41.076.710/0001-82',
    'SNEL11':'43.741.171/0001-84',
    'SNFF11':'40.011.225/0001-68',
    'SPXS11':'43.010.543/0001-00',
    'TEPP11':'26.681.370/0001-25',
    'TGAR11':'25.032.881/0001-53',
    'TOPP11':'56.805.391/0001-76',
    'TRBL11':'16.671.412/0001-93',
    'TRXF11':'28.548.288/0001-52',
    'TVRI11':'14.410.722/0001-29',
    'URPR11':'34.508.872/0001-87',
    'VCJR11':'32.400.250/0001-05',
    'VGHF11':'36.771.692/0001-19',
    'VGIP11':'34.197.811/0001-46',
    'VGIR11':'29.852.732/0001-91',
    'VGRI11':'53.656.482/0001-07',
    'VILG11':'24.853.044/0001-22',
    'VINO11':'12.516.185/0001-70',
    'VISC11':'17.554.274/0001-25',
    'VRTA11':'11.664.201/0001-00',
    'VRTM11':'51.870.412/0001-13',
    'WHGR11':'41.256.643/0001-88',
    'XPCI11':'28.516.301/0001-91',
    'XPLG11':'26.502.794/0001-85',
    'XPML11':'28.757.546/0001-00',
    'XPSF11':'30.983.020/0001-90',
}

In [4]:
# Dados mensais dos FIIs de 2022
df_fii_2022 = open_arquivos_fii(
    fii_ativo_passivo_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2022//inf_mensal_fii_ativo_passivo_2022.parquet',
    fii_complemento_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2022//inf_mensal_fii_complemento_2022.parquet',
    fii_geral_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2022//inf_mensal_fii_geral_2022.parquet'
)

# Dados mensais dos FIIs de 2023
df_fii_2023 = open_arquivos_fii(
    fii_ativo_passivo_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2023//inf_mensal_fii_ativo_passivo_2023.parquet',
    fii_complemento_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2023//inf_mensal_fii_complemento_2023.parquet',
    fii_geral_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2023//inf_mensal_fii_geral_2023.parquet'
)

# Dados mensais dos FIIs de 2024
df_fii_2024 = open_arquivos_fii(
    fii_ativo_passivo_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2024//inf_mensal_fii_ativo_passivo_2024.parquet',
    fii_complemento_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2024//inf_mensal_fii_complemento_2024.parquet',
    fii_geral_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2024//inf_mensal_fii_geral_2024.parquet'
)

# Dados mensais dos FIIs de 2025
df_fii_2025 = open_arquivos_fii(
    fii_ativo_passivo_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2025//inf_mensal_fii_ativo_passivo_2025.parquet',
    fii_complemento_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2025//inf_mensal_fii_complemento_2025.parquet',
    fii_geral_path='C://Users//vitor//projetos_python//python_b3//historico-arquivos//fundos_investimentos//fii_cvm//2025//inf_mensal_fii_geral_2025.parquet'
)

# Juntando os dfs
df_fii_compilado = pd.concat([
    df_fii_2022, 
    df_fii_2023, 
    df_fii_2024,
    df_fii_2025,
])

# Extraindo os CNPJs do 'dict_cnpj_xxx_xxx_202x'
cnpjs_mai_ago_2024 = dict_cnpj_mai_ago_2024.values()
cnpjs_set_dez_2024 = dict_cnpj_set_dez_2024.values()
cnpjs_jan_abr_2025 = dict_cnpj_jan_abr_2025.values()
cnpjs_mai_ago_2025 = dict_cnpj_mai_ago_2025.values()
cnpjs_set_dez_2025 = dict_cnpj_set_dez_2025.values()

# Selecionando apenas os FIIs que estão no IFIX
df_ifix_mai_ago_2024 = df_fii_compilado[df_fii_compilado['CNPJ_Fundo'].isin(cnpjs_mai_ago_2024)]
df_ifix_set_dez_2024 = df_fii_compilado[df_fii_compilado['CNPJ_Fundo'].isin(cnpjs_set_dez_2024)]
df_ifix_jan_abr_2025 = df_fii_compilado[df_fii_compilado['CNPJ_Fundo'].isin(cnpjs_jan_abr_2025)]
df_ifix_mai_ago_2025 = df_fii_compilado[df_fii_compilado['CNPJ_Fundo'].isin(cnpjs_mai_ago_2025)]
df_ifix_set_dez_2025 = df_fii_compilado[df_fii_compilado['CNPJ_Fundo'].isin(cnpjs_set_dez_2025)]

# Fazendo uma cópia
df_ifix_mai_ago_2024 = df_ifix_mai_ago_2024.copy() 
df_ifix_set_dez_2024 = df_ifix_set_dez_2024.copy() 
df_ifix_jan_abr_2025 = df_ifix_jan_abr_2025.copy() 
df_ifix_mai_ago_2025 = df_ifix_mai_ago_2025.copy() 
df_ifix_set_dez_2025 = df_ifix_set_dez_2025.copy() 

# Criando um dicionário reverso para mapear CNPJ (key) para ticker (value)
reverse_dict_cnpj_mai_ago_2024 = {v: k for k, v in dict_cnpj_mai_ago_2024.items()}
reverse_dict_cnpj_set_dez_2024 = {v: k for k, v in dict_cnpj_set_dez_2024.items()}
reverse_dict_cnpj_jan_abr_2025 = {v: k for k, v in dict_cnpj_jan_abr_2025.items()}
reverse_dict_cnpj_mai_ago_2025 = {v: k for k, v in dict_cnpj_mai_ago_2025.items()}
reverse_dict_cnpj_set_dez_2025 = {v: k for k, v in dict_cnpj_set_dez_2025.items()}

# Adicionando a coluna de tickers no df
df_ifix_mai_ago_2024['Ticker'] = df_ifix_mai_ago_2024['CNPJ_Fundo'].map(reverse_dict_cnpj_mai_ago_2024)
df_ifix_set_dez_2024['Ticker'] = df_ifix_set_dez_2024['CNPJ_Fundo'].map(reverse_dict_cnpj_set_dez_2024)
df_ifix_jan_abr_2025['Ticker'] = df_ifix_jan_abr_2025['CNPJ_Fundo'].map(reverse_dict_cnpj_jan_abr_2025)
df_ifix_mai_ago_2025['Ticker'] = df_ifix_mai_ago_2025['CNPJ_Fundo'].map(reverse_dict_cnpj_mai_ago_2025)
df_ifix_set_dez_2025['Ticker'] = df_ifix_set_dez_2025['CNPJ_Fundo'].map(reverse_dict_cnpj_set_dez_2025)

# Existe um fundo 'CACR11' (CNPJ:97.521.225/0001-25) que o 'Segmento_Atuacao' é 0, substituindo por 'Híbrido'
df_ifix_mai_ago_2024['Segmento_Atuacao'] = df_ifix_mai_ago_2024['Segmento_Atuacao'].replace(0, 'Híbrido')
df_ifix_set_dez_2024['Segmento_Atuacao'] = df_ifix_set_dez_2024['Segmento_Atuacao'].replace(0, 'Híbrido')
df_ifix_jan_abr_2025['Segmento_Atuacao'] = df_ifix_jan_abr_2025['Segmento_Atuacao'].replace(0, 'Híbrido')
df_ifix_mai_ago_2025['Segmento_Atuacao'] = df_ifix_mai_ago_2025['Segmento_Atuacao'].replace(0, 'Híbrido')
df_ifix_set_dez_2025['Segmento_Atuacao'] = df_ifix_set_dez_2025['Segmento_Atuacao'].replace(0, 'Híbrido')

# Selecionando a coluna 'Data_Referencia' como index
df_ifix_mai_ago_2024 = df_ifix_mai_ago_2024.set_index('Data_Referencia')
df_ifix_set_dez_2024 = df_ifix_set_dez_2024.set_index('Data_Referencia')
df_ifix_jan_abr_2025 = df_ifix_jan_abr_2025.set_index('Data_Referencia')
df_ifix_mai_ago_2025 = df_ifix_mai_ago_2025.set_index('Data_Referencia')
df_ifix_set_dez_2025 = df_ifix_set_dez_2025.set_index('Data_Referencia')

In [5]:
# Dados históricos do IFIX - de 2011 até 24-07-2025
# https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data

ifix_precos = pd.read_csv(
    'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//ifix//dados_historicos_ifix_24072025.csv',
    sep=','
)

# Removendo a coluna 'Vol.' e 'Var%'
ifix_precos = ifix_precos.drop(columns=['Vol.', 'Var%'])

# Renomeando as colunas
ifix_precos = ifix_precos.rename(columns={
    'Data': 'Date',
    'Último': 'Close',
    'Abertura': 'Open',
    'Máxima': 'High',
    'Mínima': 'Low',
})

# Arrumando os dtypes das colunas
ifix_precos['Close'] = ifix_precos['Close'].str.replace('.', '', regex=True).str.replace(',', '.', regex=True).astype(float)
ifix_precos['Open'] = ifix_precos['Open'].str.replace('.', '', regex=True).str.replace(',', '.', regex=True).astype(float)
ifix_precos['High'] = ifix_precos['High'].str.replace('.', '', regex=True).str.replace(',', '.', regex=True).astype(float)
ifix_precos['Low'] = ifix_precos['Low'].str.replace('.', '', regex=True).str.replace(',', '.', regex=True).astype(float)
ifix_precos['Date'] = pd.to_datetime(ifix_precos['Date'], format='%d.%m.%Y')

# Definindo a coluna 'Date' como index
ifix_precos = ifix_precos.set_index('Date')

# Invertendo o df
ifix_precos = ifix_precos.iloc[::-1]

# Calculando a variação percentual
ifix_precos['pct_change'] = round(ifix_precos['Close'].pct_change() * 100, 4)

# Calculando a variação percentual acumulada
ifix_precos['pct_change_accum'] = round((1 + (ifix_precos['pct_change']/100)).cumprod(), 4) 
# Primeiro dia começa em 1
ifix_precos['pct_change_accum'].iloc[0] = 1

ifix_precos

,Close,Open,High,Low,pct_change,pct_change_accum
Date,,,,,,
2011-01-03,1001.11,1001.11,1001.11,1001.11,NaN,1.0000
2011-01-04,992.56,992.56,992.56,992.56,-0.8541,0.9915
2011-01-05,998.65,998.65,998.65,998.65,0.6136,0.9975
2011-01-06,996.31,996.31,996.31,996.31,-0.2343,0.9952
2011-01-07,1000.82,1000.82,1000.82,1000.82,0.4527,0.9997
...,...,...,...,...,...,...
2025-07-18,3465.01,3475.04,3476.11,3464.55,-0.2886,3.4612
2025-07-21,3443.94,3465.01,3466.37,3443.94,-0.6081,3.4401
2025-07-22,3440.95,3443.95,3451.26,3439.28,-0.0868,3.4371


In [15]:
# Contando os números de cada segmento de atuação dos FIIs (Maio a Agosto 2024)
segmentos_fii_mai_ago_2024 = df_ifix_mai_ago_2024['Segmento_Atuacao'].value_counts()

# Contando os números de cada segmento de atuação dos FIIs (Setembro a Dezembro 2024)
segmentos_fii_set_dez_2024 = df_ifix_set_dez_2024['Segmento_Atuacao'].value_counts()

# Contando os números de cada segmento de atuação dos FIIs (Janeiro a Abril 2025)
segmentos_fii_jan_abr_2025 = df_ifix_jan_abr_2025['Segmento_Atuacao'].value_counts()

# Contando os números de cada segmento de atuação dos FIIs (Maio a Agosto 2025)
segmentos_fii_mai_ago_2025 = df_ifix_mai_ago_2025['Segmento_Atuacao'].value_counts()

# Contando os números de cada segmento de atuação dos FIIs (Setembro a Dezembro 2025)
segmentos_fii_set_dez_2025 = df_ifix_set_dez_2025['Segmento_Atuacao'].value_counts()

# Criando subplots com duas colunas
fig = make_subplots(
    rows=3, 
    cols=2, 
    specs=[
        [{'type':'domain'}, {'type':'domain'}],  # Linha 1
        [{'type':'domain'}, {'type':'domain'}],  # Linha 2
        [{'type':'domain'}, {'type':'domain'}],  # Linha 3
    ], 
    subplot_titles=[
        'Distribuição dos Segmentos dos IFIX (Maio a Agosto 2024)', 
        'Distribuição dos Segmentos dos IFIX (Setembro a Dezembro 2024)',
        'Distribuição dos Segmentos dos IFIX (Janeiro a Abril 2025)',
        'Distribuição dos Segmentos dos IFIX (Maio a Agosto 2025)',
        'Distribuição dos Segmentos dos IFIX (Setembro a Dezembro 2025)',
    ],
    vertical_spacing=0.15
)

# Adicionando o primeiro gráfico de pizza (Maio a Agosto 2024)
fig.add_trace(
    go.Pie(labels=segmentos_fii_mai_ago_2024.index, values=segmentos_fii_mai_ago_2024),
    row=1, col=1
)

# Adicionando o segundo gráfico de pizza (Setembro a Dezembro 2024)
fig.add_trace(
    go.Pie(labels=segmentos_fii_set_dez_2024.index, values=segmentos_fii_set_dez_2024),
    row=1, col=2
)

# Adicionando o terceiro gráfico de pizza (Janeiro a Abril 2025)
fig.add_trace(
    go.Pie(labels=segmentos_fii_jan_abr_2025.index, values=segmentos_fii_jan_abr_2025),
    row=2, col=1
)

# Adicionando o terceiro gráfico de pizza (Maio a Agosto 2025)
fig.add_trace(
    go.Pie(labels=segmentos_fii_mai_ago_2025.index, values=segmentos_fii_mai_ago_2025),
    row=2, col=2
)

# Adicionando o segundo gráfico de pizza (Setembro a Dezembro 2025)
fig.add_trace(
    go.Pie(labels=segmentos_fii_set_dez_2025.index, values=segmentos_fii_set_dez_2025),
    row=3, col=1
)

# Atualizando o layout
fig.update_layout(
    title_text='Distribuição dos Segmentos dos IFIX',
    title_x=0.5,  # Centralizando o título,
    width=1500,
    height=1000
)

fig.show()

In [21]:
# Ordenando em ordem alfabetica o df para ficar na mesma ordem do df do IFIX
df_ifix_alfabetico_mai_ago_2024 = df_ifix_mai_ago_2024.sort_values('Ticker')
df_ifix_alfabetico_set_dez_2024 = df_ifix_set_dez_2024.sort_values('Ticker')
df_ifix_alfabetico_jan_abr_2025 = df_ifix_jan_abr_2025.sort_values('Ticker')
df_ifix_alfabetico_mai_ago_2025 = df_ifix_mai_ago_2025.sort_values('Ticker')
df_ifix_alfabetico_set_dez_2025 = df_ifix_set_dez_2025.sort_values('Ticker')

# Selecionando os tickers do IFIX de cada quadrimestre
lst_tickers_mai_2q_2024 =  df_ifix_alfabetico_mai_ago_2024.loc['2024-05', 'Ticker']
lst_tickers_jun_2q_2024 =  df_ifix_alfabetico_mai_ago_2024.loc['2024-06', 'Ticker']
lst_tickers_jul_2q_2024 =  df_ifix_alfabetico_mai_ago_2024.loc['2024-07', 'Ticker']
lst_tickers_ago_2q_2024 =  df_ifix_alfabetico_mai_ago_2024.loc['2024-08', 'Ticker']

lst_tickers_set_3q_2024 =  df_ifix_alfabetico_set_dez_2024.loc['2024-09', 'Ticker']
lst_tickers_out_3q_2024 =  df_ifix_alfabetico_set_dez_2024.loc['2024-10', 'Ticker']
lst_tickers_nov_3q_2024 =  df_ifix_alfabetico_set_dez_2024.loc['2024-11', 'Ticker']
lst_tickers_dez_3q_2024 =  df_ifix_alfabetico_set_dez_2024.loc['2024-12', 'Ticker']

lst_tickers_jan_1q_2025 =  df_ifix_alfabetico_jan_abr_2025.loc['2025-01', 'Ticker']
lst_tickers_fev_1q_2025 =  df_ifix_alfabetico_jan_abr_2025.loc['2025-02', 'Ticker']
lst_tickers_mar_1q_2025 =  df_ifix_alfabetico_jan_abr_2025.loc['2025-03', 'Ticker']
lst_tickers_abr_1q_2025 =  df_ifix_alfabetico_jan_abr_2025.loc['2025-04', 'Ticker']

lst_tickers_mai_2q_2025 =  df_ifix_alfabetico_mai_ago_2025.loc['2025-05', 'Ticker']
lst_tickers_jun_2q_2025 =  df_ifix_alfabetico_mai_ago_2025.loc['2025-06', 'Ticker']

# Selecionandos os pesos de cada tickers do IFIX de cada quadrimestre
weight_ifix_mai_2q_2024 = ifix_mai_ago_2024.loc[ifix_mai_ago_2024['Código'].isin(lst_tickers_mai_2q_2024), 'Part. (%)']
weight_ifix_jun_2q_2024 = ifix_mai_ago_2024.loc[ifix_mai_ago_2024['Código'].isin(lst_tickers_jun_2q_2024), 'Part. (%)']
weight_ifix_jul_2q_2024 = ifix_mai_ago_2024.loc[ifix_mai_ago_2024['Código'].isin(lst_tickers_jul_2q_2024), 'Part. (%)']
weight_ifix_ago_2q_2024 = ifix_mai_ago_2024.loc[ifix_mai_ago_2024['Código'].isin(lst_tickers_ago_2q_2024), 'Part. (%)']

weight_ifix_set_3q_2024 = ifix_set_dez_2024.loc[ifix_set_dez_2024['Código'].isin(lst_tickers_set_3q_2024), 'Part. (%)']
weight_ifix_out_3q_2024 = ifix_set_dez_2024.loc[ifix_set_dez_2024['Código'].isin(lst_tickers_out_3q_2024), 'Part. (%)']
weight_ifix_nov_3q_2024 = ifix_set_dez_2024.loc[ifix_set_dez_2024['Código'].isin(lst_tickers_nov_3q_2024), 'Part. (%)']
weight_ifix_dez_3q_2024 = ifix_set_dez_2024.loc[ifix_set_dez_2024['Código'].isin(lst_tickers_dez_3q_2024), 'Part. (%)']

weight_ifix_jan_1q_2025 = ifix_jan_abr_2025.loc[ifix_jan_abr_2025['Código'].isin(lst_tickers_jan_1q_2025), 'Part. (%)']
weight_ifix_fev_1q_2025 = ifix_jan_abr_2025.loc[ifix_jan_abr_2025['Código'].isin(lst_tickers_fev_1q_2025), 'Part. (%)']
weight_ifix_mar_1q_2025 = ifix_jan_abr_2025.loc[ifix_jan_abr_2025['Código'].isin(lst_tickers_mar_1q_2025), 'Part. (%)']
weight_ifix_abr_1q_2025 = ifix_jan_abr_2025.loc[ifix_jan_abr_2025['Código'].isin(lst_tickers_abr_1q_2025), 'Part. (%)']

weight_ifix_mai_2q_2025 = ifix_mai_ago_2025.loc[ifix_mai_ago_2025['Código'].isin(lst_tickers_mai_2q_2025), 'Part. (%)']
weight_ifix_jun_2q_2025 = ifix_mai_ago_2025.loc[ifix_mai_ago_2025['Código'].isin(lst_tickers_jun_2q_2025), 'Part. (%)']

# Calculando a DY médio mensal
lst_dy_medio = []

# 2Q 2024 (mai_ago_2024)
ifix_dy_medio_mensal_202405 = np.average(df_ifix_alfabetico_mai_ago_2024.loc[f'2024-05', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_mai_2q_2024)
ifix_dy_medio_mensal_202406 = np.average(df_ifix_alfabetico_mai_ago_2024.loc[f'2024-06', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_jun_2q_2024)
ifix_dy_medio_mensal_202407 = np.average(df_ifix_alfabetico_mai_ago_2024.loc[f'2024-07', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_jul_2q_2024)
ifix_dy_medio_mensal_202408 = np.average(df_ifix_alfabetico_mai_ago_2024.loc[f'2024-08', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_ago_2q_2024)

# 3Q 2024 (set_dez_2024)
ifix_dy_medio_mensal_202409 = np.average(df_ifix_alfabetico_set_dez_2024.loc[f'2024-09', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_set_3q_2024)
ifix_dy_medio_mensal_202410 = np.average(df_ifix_alfabetico_set_dez_2024.loc[f'2024-10', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_out_3q_2024)
ifix_dy_medio_mensal_202411 = np.average(df_ifix_alfabetico_set_dez_2024.loc[f'2024-11', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_nov_3q_2024)
ifix_dy_medio_mensal_202412 = np.average(df_ifix_alfabetico_set_dez_2024.loc[f'2024-12', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_dez_3q_2024)

# 1Q 2025 (jan_abr_2025)
ifix_dy_medio_mensal_202501 = np.average(df_ifix_alfabetico_jan_abr_2025.loc[f'2025-01', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_jan_1q_2025)
ifix_dy_medio_mensal_202502 = np.average(df_ifix_alfabetico_jan_abr_2025.loc[f'2025-02', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_fev_1q_2025)
ifix_dy_medio_mensal_202503 = np.average(df_ifix_alfabetico_jan_abr_2025.loc[f'2025-03', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_mar_1q_2025)
ifix_dy_medio_mensal_202504 = np.average(df_ifix_alfabetico_jan_abr_2025.loc[f'2025-04', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_abr_1q_2025)

# 2Q 2025 (mai_ago_2025)
ifix_dy_medio_mensal_202505 = np.average(df_ifix_alfabetico_mai_ago_2025.loc[f'2025-05', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_mai_2q_2025)
ifix_dy_medio_mensal_202506 = np.average(df_ifix_alfabetico_mai_ago_2025.loc[f'2025-06', 'Percentual_Dividend_Yield_Mes'], weights=weight_ifix_jun_2q_2025)

print(f'DY médio de 05-2024 foi de {ifix_dy_medio_mensal_202405:.4f}')
print(f'DY médio de 06-2024 foi de {ifix_dy_medio_mensal_202406:.4f}')
print(f'DY médio de 07-2024 foi de {ifix_dy_medio_mensal_202407:.4f}')
print(f'DY médio de 08-2024 foi de {ifix_dy_medio_mensal_202408:.4f}')
print(f'DY médio de 09-2024 foi de {ifix_dy_medio_mensal_202409:.4f}')
print(f'DY médio de 10-2024 foi de {ifix_dy_medio_mensal_202410:.4f}')
print(f'DY médio de 11-2024 foi de {ifix_dy_medio_mensal_202411:.4f}')
print(f'DY médio de 12-2024 foi de {ifix_dy_medio_mensal_202412:.4f}')
print(f'DY médio de 01-2025 foi de {ifix_dy_medio_mensal_202501:.4f}')
print(f'DY médio de 02-2025 foi de {ifix_dy_medio_mensal_202502:.4f}')
print(f'DY médio de 03-2025 foi de {ifix_dy_medio_mensal_202503:.4f}')
print(f'DY médio de 04-2025 foi de {ifix_dy_medio_mensal_202504:.4f}')
print(f'DY médio de 05-2025 foi de {ifix_dy_medio_mensal_202505:.4f}')
print(f'DY médio de 06-2025 foi de {ifix_dy_medio_mensal_202506:.4f}')

DY médio de 05-2024 foi de 0.7941
DY médio de 06-2024 foi de 0.7971
DY médio de 07-2024 foi de 0.7659
DY médio de 08-2024 foi de 0.7473
DY médio de 09-2024 foi de 0.7793
DY médio de 10-2024 foi de 0.6872
DY médio de 11-2024 foi de 0.8241
DY médio de 12-2024 foi de 0.8867
DY médio de 01-2025 foi de 0.7983
DY médio de 02-2025 foi de 0.7641
DY médio de 03-2025 foi de 0.7742
DY médio de 04-2025 foi de 0.8154
DY médio de 05-2025 foi de 0.7941
DY médio de 06-2025 foi de 0.9160


**Selecionar o df do IFIX (atual ou anterior) p/ fazer a análise dos setores dos FIIs**

In [ ]:
# Selecionando o df para fazer a análise do setor
df_ifix = df_ifix_mai_ago_2025

### FII - Shoppings

In [ ]:
# Lista dos tickers
lst_ticker_fii_shopping = ['XPML11.SA', 'VISC11.SA', 'HGBS11.SA', 'HSML11.SA', 'MALL11.SA']

# Carregando todos os dados dos FIIs
fii_xpml = fii_cnpj(df=df_ifix, cnpj='28.757.546/0001-00', ticker='XPML11.SA')
fii_visc = fii_cnpj(df=df_ifix, cnpj='17.554.274/0001-25', ticker='VISC11.SA')
fii_hgbs = fii_cnpj(df=df_ifix, cnpj='08.431.747/0001-06', ticker='HGBS11.SA')
fii_hsml = fii_cnpj(df=df_ifix, cnpj='32.892.018/0001-31', ticker='HSML11.SA')
fii_mall = fii_cnpj(df=df_ifix, cnpj='26.499.833/0001-32', ticker='MALL11.SA')

# Mesclando todos os dfs
fii_shoppping = pd.concat([
    fii_xpml,
    fii_visc,
    fii_hgbs,
    fii_hsml,
    fii_mall
])

# Mostrando os dados do último mês divulgado
fii_shoppping.loc[
    fii_shoppping.index[-1], 
    ['Ticker', 'Close', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
].sort_values('P/VP', ascending=False)

In [ ]:
# Histórico do indicador P/VP dos FIIs 
historico_p_vp_shopping =plot_historico_p_vp(df=fii_shoppping, nome_segmento='Shopping')
historico_p_vp_shopping

In [ ]:
fii_shoppping.loc[
    fii_shoppping['Ticker'] == 'XPML11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_shoppping.loc[
    fii_shoppping['Ticker'] == 'VISC11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_shoppping.loc[
    fii_shoppping['Ticker'] == 'HGBS11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_shoppping.loc[
    fii_shoppping['Ticker'] == 'HSML11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_shoppping.loc[
    fii_shoppping['Ticker'] == 'MALL11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
# Volatilidade anual
vol_anual_shoppings = vol_anual(ticker=lst_ticker_fii_shopping, ano='2024').sort_values(ascending=False)

# Plotando a volatilidade anual
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vol_anual_shoppings.index,
    y=vol_anual_shoppings.values
))

fig.update_layout(
    title_text='Volatilidade Anual - FIIs Shoppings',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Drawdown
drawdown_shoppings = drawdown(ticker=lst_ticker_fii_shopping).sort_values(ascending=False)

# Plotando o drawndown
fig = go.Figure()

fig.add_trace(go.Bar(
    x=drawdown_shoppings.index,
    y=drawdown_shoppings.values
))

fig.update_layout(
    title_text='Drawdown - FIIs Shoppings',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Criando um df da volatilidade anual e o drawdown
df_vol_drawdown_shopping = pd.DataFrame([vol_anual_shoppings, drawdown_shoppings], index=['volatilidade_anual', 'drawndown']).T
df_vol_drawdown_shopping

In [ ]:
# Dividend yield médio, máximo e mínimo dos FII 
df_shoppings_dy = dy_fii(df=fii_shoppping)

# Plotando o dividend yield médio, máximo e mínimo
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles=[
        'Dividend Yield Médio',
        'Dividend Yield Máximo',
        'Dividend Yield Mínimo'
    ],
    vertical_spacing=0.1
    )

fig.add_trace(go.Bar(
    x=df_shoppings_dy.index,
    y=df_shoppings_dy['dy_medio'],
    name='DY médio'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=df_shoppings_dy.index,
    y=df_shoppings_dy['dy_max'],
     name='DY máximo'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=df_shoppings_dy.index,
    y=df_shoppings_dy['dy_min'],
     name='DY mínimo'
), row=3, col=1)

fig.update_layout(
    title='Dividend yield dos FII de Shopping',
    height=1200)

fig.show()

In [ ]:
# DY acumulado dos últimos 12 meses
df_shoppings_dy_accum = dy_fii_acumm_12m(df=fii_shoppping)

# Plotando o DY acumulados dos últimos 12 meses
fig = go.Figure()

for empresa in df_shoppings_dy_accum.columns:
    fig.add_trace(go.Scatter(
        x=df_shoppings_dy_accum.index,
        y=df_shoppings_dy_accum[empresa],
        name=empresa
    ))

fig.update_layout(title='Dividend yield acumulados dos últmos 12 meses - FII de Shopping')

fig.show()

In [ ]:
# Retorno anual
ret_anual_shopping = ret_anual(ticker=lst_ticker_fii_shopping, setor='Shoppings', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_anual_shopping = ret_anual_shopping[0]

# Plot do retorno anual
ret_anual_shopping[1];

In [ ]:
# Retorno acumulado
ret_acumulado_shopping = ret_acumulado(ticker=lst_ticker_fii_shopping, setor='Shoppings', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_acumulado_shopping = ret_acumulado_shopping[0]

# Plot do retorno acumulado
ret_acumulado_shopping[1];

In [ ]:
# Relação risco-retorno
plot_risk_return(ticker=lst_ticker_fii_shopping, setor='Shoppings')

**Portfólio dos Shoppings**

In [ ]:
# Lendo o arquivo do portfólio - XPML11
df_portfolio_xpml = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//portfolio_shoppings//xpml11_072025.xlsx',
)

df_portfolio_xpml

In [ ]:
# Lendo o arquivo do portfólio - VISC11
df_portfolio_visc = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//portfolio_shoppings//visc11_072025.xlsx',
)

df_portfolio_visc

In [ ]:
# Lendo o arquivo do portfólio - HGBS11
df_portfolio_hgbs = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//portfolio_shoppings//hgbs11_072025.xlsx',
    sheet_name='Portfólio',
    header=8
)

# Removendo as colunas 'Unnamed: 0' e 'Unnamed: 2'
df_portfolio_hgbs = df_portfolio_hgbs.drop(columns='Unnamed: 0')

# Removendo os NaN para retornar apenas os shoppings
df_portfolio_hgbs = df_portfolio_hgbs.dropna()

df_portfolio_hgbs

In [ ]:
# Lendo o arquivo do portfólio - HSML11
df_portfolio_hsml = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//portfolio_shoppings//hsml11_072025.xlsx',
)

df_portfolio_hsml

In [ ]:
# Lendo o arquivo do portfólio - MALL11
df_portfolio_mall = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//portfolio_shoppings//mall11_052025.xlsx',
)

df_portfolio_mall

* ABL: é um indicador que mede a área total de um shopping center que está disponível para aluguel, exceto os quiosques.

In [ ]:
# Transformando em um arquivo excel
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//fii//fii_shoppings.xlsx') as writer:
    fii_xpml.to_excel(writer, sheet_name='XPML11')
    fii_visc.to_excel(writer, sheet_name='VISC11')
    fii_hgbs.to_excel(writer, sheet_name='HGBS11')
    fii_hsml.to_excel(writer, sheet_name='HSML11')
    fii_mall.to_excel(writer, sheet_name='MALL11')
    df_shoppings_dy.to_excel(writer, sheet_name='DIVIDEND_YIELD')
    df_shoppings_dy_accum.to_excel(writer, sheet_name='DIVIDEND_YIELD_ACUM')
    df_vol_drawdown_shopping.to_excel(writer, sheet_name='VOLATILIDADE_DRAWDOWN')
    df_ret_anual_shopping.to_excel(writer, sheet_name='RET_ANUAL')
    df_ret_acumulado_shopping.to_excel(writer, sheet_name='RET_ACUM')

**Indicador P/VP das ações de shoppings**
* Atualizar o 'dict_acoes_shoppings' para calcular o P/VP das empresas de shoppings;
---
* Quando o tipo da contabilidade adotada pelo shopping é:
    * **mercado**: o valor do 'propriedades_investimento' e 'valor_justo' são iguais;
        * Esse valor é encontrado no balanço patrimonal na parte do ativo é o item 'Propriedades para Investimento'.
    * **custo**: o valor do 'propriedades_investimento' e 'valor_justo' são diferentes.
        * O valor do 'propriedades_investimento' é encontrado no balanço patrimonal na parte do ativo é o item 'Propriedades para Investimento';
        * O valor do 'valor_justo' é encontrado na nota explicativa "propriedades para investimento".


In [ ]:
# Ações de shoppings
lst_acoes_shoppings = ['ALOS3.SA', 'IGTI11.SA', 'MULT3.SA']

# Dados do 1T2024
dict_acoes_shoppings_1T24 = {
    'tipo_contabilidade': [
        'mercado',  # ALOS3 
        'custo',    # IGTI11
        'custo'     # MULT3
    ], 

    'num_acoes':[
        573936909,  # ALOS3 
        1233106910, # IGTI11
        600760875   # MULT3
    ],

    'propriedades_investimento':[
        21363200000,  # ALOS3 
        5066710000,   # IGTI11
        83183552000   # MULT3
    ],

    'valor_justo':[
        21363200000,  # ALOS3 
        14069612000,  # IGTI11
        29200692000   # MULT3
    ],

    'pl_contabil':[
        15006995000,  # ALOS3 
        4234679000,   # IGTI11
        7087747000    # MULT3
    ],

    'abl_propria':[
        1328809,  # ALOS3  
        491270,   # IGTI11
        713765    # MULT3
    ],

    'divida_liquida':[
        3526530000,  # ALOS3  
        1735990000,  # IGTI11
        2042648000   # MULT3
    ]
}

# Dados do 2T2024
dict_acoes_shoppings_2T24 = {
    'tipo_contabilidade': [
        'mercado',  # ALOS3 
        'custo',    # IGTI11
        'custo'     # MULT3
    ], 

    'num_acoes':[
        542936909,  # ALOS3 
        1217931130, # IGTI11
        600760875   # MULT3
    ],

    'propriedades_investimento':[
        20569241000,  # ALOS3 
        4987082000,   # IGTI11
        8553030000    # MULT3
    ],

    'valor_justo':[
        20569241000,  # ALOS3  
        14069612000,  # IGTI11
        30404491000   # MULT3
    ],

    'pl_contabil':[
        14888005000,  # ALOS3  
        4159817000,   # IGTI11
        7164748000    # MULT3
    ],

    'abl_propria':[
        1276678,  # ALOS3 
        491431,   # IGTI11
        758820    # MULT3
    ],

    'divida_liquida':[
        3003939000,  # ALOS3 
        1741906000,  # IGTI11
        2193197000   # MULT3
    ]
}

# Dados do 3T2024
dict_acoes_shoppings_3T24 = {
    'tipo_contabilidade': [
        'mercado', # ALOS3 
        'custo',   # IGTI11
        'custo'    # MULT3
    ], 

    'num_acoes':[
        542936909,  # ALOS3 
        1217931130, # IGTI11
        578163701   # MULT3
    ],

    'propriedades_investimento':[
        20556395000, # ALOS3 
        5452522000,  # IGTI11
        8686208000   # MULT3
    ],

    'valor_justo':[
        20556395000, # ALOS3 
        13827886000, # IGTI11
        30834077000  # MULT3
    ],

    'pl_contabil':[
        14710918000, # ALOS3  
        4263948000, # IGTI11
        7340951000  # MULT3
    ],

    'abl_propria':[
        1291465, # ALOS3   
        471897, # IGTI11 
        758744  # MULT3
    ],
    
    'divida_liquida':[
        3290524000, # ALOS3    
        1621387000, # IGTI11 
        2216800000  # MULT3
    ]
}

# Dados do DFP2024
dict_acoes_shoppings_4T24 = {
    'tipo_contabilidade': [
        'mercado', # ALOS3 
        'custo',   # IGTI11
        'custo'    # MULT3
    ], 

    'num_acoes':[
        542936909,  # ALOS3 
        1217931130, # IGTI11
        519163701   # MULT3
    ],

    'propriedades_investimento':[
        20700140000, # ALOS3 
        5520922000,  # IGTI11
        8909922000   # MULT3
    ],

    'valor_justo':[
        20700140000, # ALOS3 
        15631045000, # IGTI11
        30093435000  # MULT3
    ],

    'pl_contabil':[
        14134520000, # ALOS3  
        4310020000, # IGTI11
        5645769000  # MULT3
    ],

    'abl_propria':[
        1287355, # ALOS3   
        480968, # IGTI11 
        765102  # MULT3
    ],

    'divida_liquida':[
        3671487000, # ALOS3    
        1614844000, # IGTI11 
        4274600000  # MULT3
    ]
}

# Dados do 1T2025
dict_acoes_shoppings_1T25 = {
    'tipo_contabilidade': [
        'mercado',  # ALOS3 
        'custo',    # IGTI11
        'custo'     # MULT3
    ], 

    'num_acoes':[
        542_936_909,  # ALOS3 
        1_206_361_185, # IGTI11
        513_163_701   # MULT3
    ],

    'propriedades_investimento':[
        20_253_531_000,  # ALOS3 
        5_543_287_000,   # IGTI11
        8_989_257_000   # MULT3
    ],

    'valor_justo':[
        20_253_531_000,  # ALOS3 
        15_631_045_000,  # IGTI11
        31_355_333_000   # MULT3
    ],

    'pl_contabil':[
        12_198_264_000,  # ALOS3 
        4_409_459_000,   # IGTI11
        5_746_760_000    # MULT3
    ],

    'abl_propria':[
        1_254_323,  # ALOS3  
        479_246,   # IGTI11
        713_765    # MULT3
    ],

    'divida_liquida':[
        3_516_384_000,  # ALOS3  
        1_840_418_000,  # IGTI11
        4_231_300_000   # MULT3
    ]
}

# Dados do 2T2025
dict_acoes_shoppings_2T25 = {
    'tipo_contabilidade': [
        'mercado',  # ALOS3 
        'custo',    # IGTI11
        'custo'     # MULT3
    ], 

    'num_acoes':[
        542_936_909,  # ALOS3 
        1_206_361_185, # IGTI11
        513_163_701   # MULT3
    ],

    'propriedades_investimento':[
        20_235_584_000,  # ALOS3 
        6_051_359_000,   # IGTI11
        9_103_581_000   # MULT3
    ],

    'valor_justo':[
        20_235_584_000,  # ALOS3 
        15_762_431_000,  # IGTI11
        31_894_339_000   # MULT3
    ],

    'pl_contabil':[
        14_209_277_000,  # ALOS3 
        4_514_444_000,   # IGTI11
        5_905_361_000    # MULT3
    ],

    'abl_propria':[
        1_242_280,  # ALOS3  
        489_095,   # IGTI11
        765_466    # MULT3
    ],

    'divida_liquida':[
        3_418_573_000,  # ALOS3  
        2_381_817_000,  # IGTI11
        4_382_100_000   # MULT3
    ]
}

# 1T2024
df_acoes_shoppings_1T24 = indicadores_acoes_shoppings(
    ano='2024', 
    mes='03', 
    dict_shoppings=dict_acoes_shoppings_1T24, 
    ticker=lst_acoes_shoppings, 
    vm_igti=vm_igti(ano='2024', mes='03', num_on=797612750, num_pn=435494160)
)

# 2T2024
df_acoes_shoppings_2T24 = indicadores_acoes_shoppings(
    ano='2024', 
    mes='06', 
    dict_shoppings=dict_acoes_shoppings_2T24, 
    ticker=lst_acoes_shoppings, 
    vm_igti=vm_igti(ano='2024', mes='06', num_on=774849080, num_pn=443082050)
)

# 3T2024
df_acoes_shoppings_3T24 = indicadores_acoes_shoppings(
    ano='2024', 
    mes='09', 
    dict_shoppings=dict_acoes_shoppings_3T24, 
    ticker=lst_acoes_shoppings, 
    vm_igti=vm_igti(ano='2024', mes='09', num_on=774849080, num_pn=443082050)
)

# 4T2024
df_acoes_shoppings_4T24 = indicadores_acoes_shoppings(
    ano='2024', 
    mes='12', 
    dict_shoppings=dict_acoes_shoppings_4T24, 
    ticker=lst_acoes_shoppings, 
    vm_igti=vm_igti(ano='2024', mes='12', num_on=774849080, num_pn=443082050)
)

# 1T2025
df_acoes_shoppings_1T25 = indicadores_acoes_shoppings(
    ano='2025', 
    mes='03', 
    dict_shoppings=dict_acoes_shoppings_1T25, 
    ticker=lst_acoes_shoppings, 
    vm_igti=vm_igti(ano='2025', mes='03', num_on=770992429, num_pn=435368756)
)

# 2T2025
df_acoes_shoppings_2T25 = indicadores_acoes_shoppings(
    ano='2025', 
    mes='06', 
    dict_shoppings=dict_acoes_shoppings_2T25, 
    ticker=lst_acoes_shoppings, 
    vm_igti=vm_igti(ano='2025', mes='06', num_on=770992429, num_pn=435368756)
)

In [ ]:
# Df do valor de mercado
df_shopping_valor_mercado = pd.concat([
    df_acoes_shoppings_1T24['valor_mercado'],
    df_acoes_shoppings_2T24['valor_mercado'],
    df_acoes_shoppings_3T24['valor_mercado'],
    df_acoes_shoppings_4T24['valor_mercado'],
    df_acoes_shoppings_1T25['valor_mercado'],
], axis=1)

# Renomeando as colunas 
df_shopping_valor_mercado.columns = [
    '1T24', '2T24', '3T24', '4T24',
    '1T25'
]

# Transpondo o df
df_shopping_valor_mercado = df_shopping_valor_mercado.T

# Plotando o valor de mercado
fig = go.Figure()

for idx, col in enumerate(df_shopping_valor_mercado.columns):
    fig.add_trace(go.Bar(
        x=df_shopping_valor_mercado.index,
        y=df_shopping_valor_mercado[col],
        name=col
    ))

fig.update_layout(
    title_text='Valor de Mercado - Shoppings',
    template='seaborn'
)

fig.show()

In [ ]:
# Df do valor justo das propriedades para investimento
df_shopping_valor_justo = pd.concat([
    df_acoes_shoppings_1T24['valor_justo'],
    df_acoes_shoppings_2T24['valor_justo'],
    df_acoes_shoppings_3T24['valor_justo'],
    df_acoes_shoppings_4T24['valor_justo'],
    df_acoes_shoppings_1T25['valor_justo'],
], axis=1)

# Renomeando as colunas 
df_shopping_valor_justo.columns = [
    '1T24', '2T24', '3T24', '4T2024',
    '1T25'
]

# Transpondo o df
df_shopping_valor_justo = df_shopping_valor_justo.T

# Plotando o valor justo
fig = go.Figure()

for idx, col in enumerate(df_shopping_valor_justo.columns):
    fig.add_trace(go.Bar(
        x=df_shopping_valor_justo.index,
        y=df_shopping_valor_justo[col],
        name=col
    ))

fig.update_layout(
    title_text='Valor Justo - Shoppings',
    template='seaborn'
)

fig.show()

In [ ]:
# Df do P/VP dos shoppings
df_shopping_p_vp = pd.concat([
    round(df_acoes_shoppings_1T24['p/vp'], 2),
    round(df_acoes_shoppings_2T24['p/vp'], 2),
    round(df_acoes_shoppings_3T24['p/vp'], 2),
    round(df_acoes_shoppings_4T24['p/vp'], 2),
    round(df_acoes_shoppings_1T25['p/vp'], 2),
], axis=1)

# Renomeando as colunas 
df_shopping_p_vp.columns = [
    '1T24', '2T24', '3T24', '4T24',
    '1T25'
]

# Transpondo o df
df_shopping_p_vp = df_shopping_p_vp.T

# Plotando o P/VP dos shoppings
fig = go.Figure()

for idx, col in enumerate(df_shopping_p_vp.columns):
    fig.add_trace(go.Scatter(
        x=df_shopping_p_vp.index,
        y=df_shopping_p_vp[col],
        name=col
    ))

fig.add_hline(
    y=0,  
    line_dash='dash', 
    line_color='red', 
    line_width=2  
)

fig.update_layout(
    title_text='Indicador P/VP - Shoppings',
    template='seaborn'
)

fig.show()

In [ ]:
# Df do EV/ABL dos shoppings
df_shopping_ev_abl = pd.concat([
    round(df_acoes_shoppings_1T24['ev/abl'], 2),
    round(df_acoes_shoppings_2T24['ev/abl'], 2),
    round(df_acoes_shoppings_3T24['ev/abl'], 2),
    round(df_acoes_shoppings_4T24['ev/abl'], 2),
    round(df_acoes_shoppings_1T25['ev/abl'], 2),
], axis=1)

# Renomeando as colunas 
df_shopping_ev_abl.columns = [
    '1T24', '2T24', '3T24', '4T24',
    '1T25'
]

# Transpondo o df
df_shopping_ev_abl = df_shopping_ev_abl.T

# Plotando o EV/ABL dos shoppings
fig = go.Figure()

for idx, col in enumerate(df_shopping_ev_abl.columns):
    fig.add_trace(go.Scatter(
        x=df_shopping_ev_abl.index,
        y=df_shopping_ev_abl[col],
        name=col
    ))

fig.update_layout(
    title_text='Indicador EV/ABL - Shoppings',
    template='seaborn'
)

fig.show()

### FII - Logístico/Industrial/Galpões

In [ ]:
# Lista dos tickers
lst_ticker_fii_logistica = [
    'HGLG11.SA', 
    'BTLG11.SA',
    'XPLG11.SA',
    'LVBI11.SA',
    'BRCO11.SA',
    'TRBL11.SA',
    'KNRI11.SA'
]

# Carregando todos os dados dos FIIs
fii_hglg = fii_cnpj(df=df_ifix, cnpj='11.728.688/0001-47', ticker='HGLG11.SA')
fii_btlg = fii_cnpj(df=df_ifix, cnpj='11.839.593/0001-09', ticker='BTLG11.SA')
fii_xplg = fii_cnpj(df=df_ifix, cnpj='26.502.794/0001-85', ticker='XPLG11.SA')
fii_lvbi = fii_cnpj(df=df_ifix, cnpj='30.629.603/0001-18', ticker='LVBI11.SA')
fii_brco = fii_cnpj(df=df_ifix, cnpj='20.748.515/0001-81', ticker='BRCO11.SA')
fii_trbl = fii_cnpj(df=df_ifix, cnpj='16.671.412/0001-93', ticker='TRBL11.SA')
fii_knri = fii_cnpj(df=df_ifix, cnpj='12.005.956/0001-65', ticker='KNRI11.SA')

# Mesclando todos os últimos dados
fii_logistica = pd.concat([
    fii_hglg,
    fii_btlg,
    fii_xplg,
    fii_lvbi,
    fii_brco,
    fii_trbl,
    fii_knri
])

# Mostrando os dados do último mês divulgado
fii_logistica.loc[
    fii_logistica.index[-1],
    ['Ticker', 'Close', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
].sort_values('P/VP', ascending=False)

In [ ]:
# Histórico do indicador P/VP dos FIIs 
historico_p_vp_logistica =plot_historico_p_vp(df=fii_logistica, nome_segmento='Logística')
historico_p_vp_logistica

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'HGLG11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'BTLG11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'XPLG11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'LVBI11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'BRCO11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'TRBL11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_logistica.loc[
    fii_logistica['Ticker'] == 'KNRI11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
# Volatilidade anual
vol_anual_logistica = vol_anual(ticker=lst_ticker_fii_logistica, ano='2024').sort_values(ascending=False)

# Plotando a volatilidade anual
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vol_anual_logistica.index,
    y=vol_anual_logistica.values
))

fig.update_layout(
    title_text='Volatilidade Anual - FIIs de Logística',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Drawdown
drawdown_logistica = drawdown(ticker=lst_ticker_fii_logistica).sort_values(ascending=False)

# Plotando o drawndown
fig = go.Figure()

fig.add_trace(go.Bar(
    x=drawdown_logistica.index,
    y=drawdown_logistica.values
))

fig.update_layout(
    title_text='Drawdown - FIIs Galpões Logísticos',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Criando um df da volatilidade anual e o drawdown
df_vol_drawdown_logistica = pd.DataFrame([vol_anual_logistica, drawdown_logistica], index=['volatilidade_anual', 'drawndown']).T
df_vol_drawdown_logistica

In [ ]:
# Dividend yield médio, máximo e mínimo dos FII 
df_logistica_dy = dy_fii(df=fii_logistica)

# Plotando o dividend yield médio, máximo e mínimo
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles=[
        'Dividend Yield Médio',
        'Dividend Yield Máximo',
        'Dividend Yield Mínimo'
    ],
    vertical_spacing=0.1
    )

fig.add_trace(go.Bar(
    x=df_logistica_dy.index,
    y=df_logistica_dy['dy_medio'],
    name='DY médio'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=df_logistica_dy.index,
    y=df_logistica_dy['dy_max'],
     name='DY máximo'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=df_logistica_dy.index,
    y=df_logistica_dy['dy_min'],
     name='DY mínimo'
), row=3, col=1)

fig.update_layout(
    title='Dividend yield dos FII de Logística',
    height=1200)

fig.show()

In [ ]:
# DY acumulado dos últimos 12 meses
df_logistica_dy_accum = dy_fii_acumm_12m(df=fii_logistica)

# Plotando o DY acumulados dos últimos 12 meses
fig = go.Figure()

for empresa in df_logistica_dy_accum.columns:
    fig.add_trace(go.Scatter(
        x=df_logistica_dy_accum.index,
        y=df_logistica_dy_accum[empresa],
        name=empresa
    ))

fig.update_layout(title='Dividend yield acumulados dos últmos 12 meses - FII de Logística')

fig.show()

In [ ]:
# Retorno anual
ret_anual_logistica = ret_anual(ticker=lst_ticker_fii_logistica, setor='Galpões Logísticos', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_anual_logistica = ret_anual_logistica[0]

# Plot do retorno anual
ret_anual_logistica[1];

In [ ]:
# Retorno acumulado
ret_acumulado_logistica = ret_acumulado(ticker=lst_ticker_fii_logistica, setor='Galpões Logísticos', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_acumulado_logistica = ret_acumulado_logistica[0]

# Plot do retorno anual
ret_acumulado_logistica[1];

In [ ]:
# Relação risco-retorno
plot_risk_return(ticker=lst_ticker_fii_logistica, setor='Galpões Logísticos')

In [ ]:
# Transformando em um arquivo excel
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//fii//fii_logistica.xlsx') as writer:
    fii_hglg.to_excel(writer, sheet_name='HGLG11')
    fii_btlg.to_excel(writer, sheet_name='BTLG11')
    fii_xplg.to_excel(writer, sheet_name='XPLG11')
    fii_lvbi.to_excel(writer, sheet_name='LVBI11')
    fii_brco.to_excel(writer, sheet_name='BRCO11')
    fii_trbl.to_excel(writer, sheet_name='TRBL11')
    fii_knri.to_excel(writer, sheet_name='KNRI11')
    df_logistica_dy.to_excel(writer, sheet_name='DIVIDEND_YIELD')
    df_logistica_dy_accum.to_excel(writer, sheet_name='DIVIDEND_YIELD_ACUM')
    df_vol_drawdown_logistica.to_excel(writer, sheet_name='VOLATILIDADE_DRAWDOWN')
    df_ret_anual_logistica.to_excel(writer, sheet_name='RET_ANUAL')
    df_ret_acumulado_logistica.to_excel(writer, sheet_name='RET_ACUM')

### FII - Lajes Corporativas

In [ ]:
# Lista dos tickers
lst_ticker_fii_lajes = ['PVBI11.SA', 'HGPO11.SA', 'KORE11.SA']

# Carregando todos os dados dos FIIs
fii_pvbi = fii_cnpj(df=df_ifix, cnpj='35.652.102/0001-76', ticker='PVBI11.SA')
fii_hgpo = fii_cnpj(df=df_ifix, cnpj='11.260.134/0001-68', ticker='HGPO11.SA')
fii_kore = fii_cnpj(df=df_ifix, cnpj='52.219.978/0001-42', ticker='KORE11.SA')

# Mesclando todos os últimos dados
fii_lajes = pd.concat([
    fii_pvbi,
    fii_hgpo,
    fii_kore
])

# Mostrando os dados do último mês divulgado
fii_lajes.loc[
    fii_lajes.index[-1],
    ['Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes'] 
].sort_values('P/VP', ascending=False)

In [ ]:
# Histórico do indicador P/VP dos FIIs 
historico_p_vp_lajes =plot_historico_p_vp(df=fii_lajes, nome_segmento='Lajes Corporativas')
historico_p_vp_lajes

In [ ]:
fii_lajes.loc[
    fii_lajes['Ticker'] == 'PVBI11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_lajes.loc[
    fii_lajes['Ticker'] == 'HGPO11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_lajes.loc[
    fii_lajes['Ticker'] == 'KORE11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
# Volatilidade anual
vol_anual_lajes = vol_anual(ticker=lst_ticker_fii_lajes, ano='2024').sort_values(ascending=False)

# Plotando a volatilidade anual
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vol_anual_lajes.index,
    y=vol_anual_lajes.values
))

fig.update_layout(
    title_text='Volatilidade Anual - FIIs Lajes Corporativas',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Drawdown
drawdown_lajes = drawdown(ticker=lst_ticker_fii_lajes).sort_values(ascending=False)

# Plotando o drawndown
fig = go.Figure()

fig.add_trace(go.Bar(
    x=drawdown_lajes.index,
    y=drawdown_lajes.values
))

fig.update_layout(
    title_text='Drawdown - FIIs Lajes Corporativas',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Criando um df da volatilidade anual e o drawdown
df_vol_drawdown_lajes = pd.DataFrame([vol_anual_lajes, drawdown_lajes], index=['volatilidade_anual', 'drawndown']).T
df_vol_drawdown_lajes

In [ ]:
# Dividend yield médio, máximo e mínimo dos FII 
df_lajes_dy = dy_fii(df=fii_lajes)

# Plotando o dividend yield médio, máximo e mínimo
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles=[
        'Dividend Yield Médio',
        'Dividend Yield Máximo',
        'Dividend Yield Mínimo'
    ],
    vertical_spacing=0.1
    )

fig.add_trace(go.Bar(
    x=df_lajes_dy.index,
    y=df_lajes_dy['dy_medio'],
    name='DY médio'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=df_lajes_dy.index,
    y=df_lajes_dy['dy_max'],
     name='DY máximo'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=df_lajes_dy.index,
    y=df_lajes_dy['dy_min'],
     name='DY mínimo'
), row=3, col=1)

fig.update_layout(
    title='Dividend yield dos FII de Lajes Corporativas',
    height=1200)

fig.show()

In [ ]:
# DY acumulado dos últimos 12 meses
df_lajes_dy_accum = dy_fii_acumm_12m(df=fii_lajes)

# Plotando o DY acumulados dos últimos 12 meses
fig = go.Figure()

for empresa in df_lajes_dy_accum.columns:
    fig.add_trace(go.Scatter(
        x=df_lajes_dy_accum.index,
        y=df_lajes_dy_accum[empresa],
        name=empresa
    ))

fig.update_layout(title='Dividend yield acumulados dos últmos 12 meses - FII de Lajes Corporativas')

fig.show()

In [ ]:
# Retorno anual
ret_anual_lajes = ret_anual(ticker=lst_ticker_fii_lajes, setor='Lajes Corporativas', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_anual_lajes = ret_anual_lajes[0]

# Plot do retorno anual
ret_anual_lajes[1];

In [ ]:
# Retorno acumulado
ret_acumulado_lajes = ret_acumulado(ticker=lst_ticker_fii_lajes, setor='Lajes Corporativas', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_acumulado_lajes = ret_acumulado_lajes[0]

# Plot do retorno anual
ret_acumulado_lajes[1];

In [ ]:
# Relação risco-retorno
plot_risk_return(ticker=lst_ticker_fii_lajes, setor='Lajes Corporativas')

In [ ]:
# Transformando em um arquivo excel
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//fii//fii_lajes.xlsx') as writer:
    fii_pvbi.to_excel(writer, sheet_name='PVBI11')
    fii_hgpo.to_excel(writer, sheet_name='HGPO11')
    fii_kore.to_excel(writer, sheet_name='KORE11')
    df_lajes_dy.to_excel(writer, sheet_name='DIVIDEND_YIELD')
    df_lajes_dy_accum.to_excel(writer, sheet_name='DIVIDEND_YIELD_ACUM')
    df_vol_drawdown_lajes.to_excel(writer, sheet_name='VOLATILIDADE_DRAWDOWN')
    df_ret_anual_lajes.to_excel(writer, sheet_name='RET_ANUAL')
    df_ret_acumulado_lajes.to_excel(writer, sheet_name='RET_ACUM')

### FI - Renda Urbana

In [ ]:
# Lista dos tickers
lst_ticker_fii_renda_urbana = ['HGRU11.SA', 'TRXF11.SA', 'VIUR11.SA', 'RBVA11.SA']

# Carregando todos os dados dos FIIs
fii_hgru = fii_cnpj(df=df_ifix, cnpj='29.641.226/0001-53', ticker='HGRU11.SA')
fii_trxf = fii_cnpj(df=df_ifix, cnpj='28.548.288/0001-52', ticker='TRXF11.SA')
fii_viur = fii_cnpj(df=df_ifix, cnpj='36.445.587/0001-90', ticker='VIUR11.SA')
fii_rbva = fii_cnpj(df=df_ifix, cnpj='15.576.907/0001-70', ticker='RBVA11.SA')

# Mesclando todos os últimos dados
fii_renda_urbana = pd.concat([
    fii_hgru,
    fii_trxf,
    fii_viur,
    fii_rbva
])

# Mostrando os dados do últimos mês divulgado
fii_renda_urbana.loc[
    fii_renda_urbana.index[-1],
    ['Ticker', 'Close', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
].sort_values('P/VP', ascending=False)

In [ ]:
# Histórico do indicador P/VP dos FIIs 
historico_p_vp_renda_urbana = plot_historico_p_vp(df=fii_renda_urbana, nome_segmento='Renda Urbana')
historico_p_vp_renda_urbana

In [ ]:
fii_renda_urbana.loc[
    fii_renda_urbana['Ticker'] == 'HGRU11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_renda_urbana.loc[
    fii_renda_urbana['Ticker'] == 'TRXF11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_renda_urbana.loc[
    fii_renda_urbana['Ticker'] == 'VIUR11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_renda_urbana.loc[
    fii_renda_urbana['Ticker'] == 'RBVA11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
# Volatilidade anual
vol_anual_renda_urbana = vol_anual(ticker=lst_ticker_fii_renda_urbana, ano='2024').sort_values(ascending=False)

# Plotando a volatilidade anual
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vol_anual_renda_urbana.index,
    y=vol_anual_renda_urbana.values
))

fig.update_layout(
    title_text='Volatilidade Anual - FIIs Renda Urbana',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Drawdown
drawdown_renda_urbana = drawdown(ticker=lst_ticker_fii_renda_urbana).sort_values(ascending=False)

# Plotando o drawndown
fig = go.Figure()

fig.add_trace(go.Bar(
    x=drawdown_renda_urbana.index,
    y=drawdown_renda_urbana.values
))

fig.update_layout(
    title_text='Drawdown - FIIs Renda Urbana',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Criando um df da volatilidade anual e o drawdown
df_vol_drawdown_renda_urbana = pd.DataFrame([vol_anual_renda_urbana, drawdown_renda_urbana], index=['volatilidade_anual', 'drawndown']).T
df_vol_drawdown_renda_urbana

In [ ]:
# Dividend yield médio, máximo e mínimo dos FII 
df_renda_urbana_dy = dy_fii(df=fii_renda_urbana)

# Plotando o dividend yield médio, máximo e mínimo
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles=[
        'Dividend Yield Médio',
        'Dividend Yield Máximo',
        'Dividend Yield Mínimo'
    ],
    vertical_spacing=0.1
    )

fig.add_trace(go.Bar(
    x=df_renda_urbana_dy.index,
    y=df_renda_urbana_dy['dy_medio'],
    name='DY médio'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=df_renda_urbana_dy.index,
    y=df_renda_urbana_dy['dy_max'],
     name='DY máximo'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=df_renda_urbana_dy.index,
    y=df_renda_urbana_dy['dy_min'],
     name='DY mínimo'
), row=3, col=1)

fig.update_layout(
    title='Dividend yield dos FII de Renda Urbana',
    height=1200)

fig.show()

In [ ]:
# DY acumulado dos últimos 12 meses
df_renda_urbana_dy_accum = dy_fii_acumm_12m(df=fii_renda_urbana)

# Plotando o DY acumulados dos últimos 12 meses
fig = go.Figure()

for empresa in df_renda_urbana_dy_accum.columns:
    fig.add_trace(go.Scatter(
        x=df_renda_urbana_dy_accum.index,
        y=df_renda_urbana_dy_accum[empresa],
        name=empresa
    ))

fig.update_layout(title='Dividend yield acumulados dos últmos 12 meses - FII de Renda Urbana')

fig.show()

In [ ]:
# Retorno anual
ret_anual_renda_urbana = ret_anual(ticker=lst_ticker_fii_renda_urbana, setor='Renda Urbana', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_anual_renda_urbana = ret_anual_renda_urbana[0]

# Plot do retorno anual
ret_anual_renda_urbana[1];

In [ ]:
# Retorno acumulado
ret_acumulado_renda_urbana = ret_acumulado(ticker=lst_ticker_fii_renda_urbana, setor='Renda Urbana', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_acumulado_renda_urbana = ret_acumulado_renda_urbana[0]

# Plot do retorno anual
ret_acumulado_renda_urbana[1];

In [ ]:
# Relação risco-retorno
plot_risk_return(ticker=lst_ticker_fii_renda_urbana, setor='Renda Urbana')

In [ ]:
# Transformando em um arquivo excel
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//fii//fii_renda_urbana.xlsx') as writer:
    fii_hgru.to_excel(writer, sheet_name='HGRU11')
    fii_trxf.to_excel(writer, sheet_name='TRXF11')
    fii_viur.to_excel(writer, sheet_name='VIUR11')
    fii_rbva.to_excel(writer, sheet_name='RBVA11')
    df_renda_urbana_dy.to_excel(writer, sheet_name='DIVIDEND_YIELD')
    df_renda_urbana_dy_accum.to_excel(writer, sheet_name='DIVIDEND_YIELD_ACUM')
    df_vol_drawdown_renda_urbana.to_excel(writer, sheet_name='VOLATILIDADE_DRAWDOWN')
    df_ret_anual_renda_urbana.to_excel(writer, sheet_name='RET_ANUAL')
    df_ret_acumulado_renda_urbana.to_excel(writer, sheet_name='RET_ACUM')

### FI - Papel

In [ ]:
# Lista dos tickers
lst_ticker_fii_papel = ['KNIP11.SA', 'KNCR11.SA', 'KNSC11.SA', 'HGCR11.SA', 'MXRF11.SA', 'VGIR11.SA']

# Carregando todos os dados dos FIIs
fii_knip = fii_cnpj(df=df_ifix, cnpj='24.960.430/0001-13', ticker='KNIP11.SA')
fii_kncr = fii_cnpj(df=df_ifix, cnpj='16.706.958/0001-32', ticker='KNCR11.SA')
fii_knsc = fii_cnpj(df=df_ifix, cnpj='35.864.448/0001-38', ticker='KNSC11.SA')
fii_hgcr = fii_cnpj(df=df_ifix, cnpj='11.160.521/0001-22', ticker='HGCR11.SA')
fii_mxrf = fii_cnpj(df=df_ifix, cnpj='97.521.225/0001-25', ticker='MXRF11.SA')
fii_vgir = fii_cnpj(df=df_ifix, cnpj='29.852.732/0001-91', ticker='VGIR11.SA')

# Mesclando todos os últimos dados
fii_papel = pd.concat([
    fii_knip,
    fii_kncr,
    fii_knsc,
    fii_hgcr,
    fii_mxrf,
    fii_vgir
])

# Mostrando os dados do último mês divulgado
fii_papel.loc[
    fii_papel.index[-1],
    ['Ticker', 'Close', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
].sort_values('P/VP', ascending=False)

In [ ]:
# Histórico do indicador P/VP dos FIIs 
historico_p_vp_papel = plot_historico_p_vp(df=fii_papel, nome_segmento='Papel')
historico_p_vp_papel

In [ ]:
fii_papel.loc[
    fii_papel['Ticker'] == 'KNIP11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_papel.loc[
    fii_papel['Ticker'] == 'KNCR11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_papel.loc[
    fii_papel['Ticker'] == 'KNSC11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_papel.loc[
    fii_papel['Ticker'] == 'HGCR11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_papel.loc[
    fii_papel['Ticker'] == 'MXRF11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
fii_papel.loc[
    fii_papel['Ticker'] == 'VGIR11', 
    ['CNPJ_Fundo', 'Ticker', 'Close', 'Divida_Total', 'Grau_Alavancagem', 'P/VP', 'Percentual_Rentabilidade_Efetiva_Mes', 'Percentual_Dividend_Yield_Mes']
]

In [ ]:
# Volatilidade anual
vol_anual_papel = vol_anual(ticker=lst_ticker_fii_papel, ano='2024').sort_values(ascending=False)

# Plotando a volatilidade anual
fig = go.Figure()

fig.add_trace(go.Bar(
    x=vol_anual_papel.index,
    y=vol_anual_papel.values
))

fig.update_layout(
    title_text='Volatilidade Anual - FIIs Papel',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Drawdown
drawdown_papel = drawdown(ticker=lst_ticker_fii_papel).sort_values(ascending=False)

# Plotando o drawndown
fig = go.Figure()

fig.add_trace(go.Bar(
    x=drawdown_papel.index,
    y=drawdown_papel.values
))

fig.update_layout(
    title_text='Drawdown - FIIs Papel',
    template='seaborn',
    showlegend=False   
)

fig.show()

In [ ]:
# Criando um df da volatilidade anual e o drawdown
df_vol_drawdown_papel = pd.DataFrame([vol_anual_papel, drawdown_papel], index=['volatilidade_anual', 'drawndown']).T
df_vol_drawdown_papel

In [ ]:
# Dividend yield médio, máximo e mínimo dos FII 
df_papel_dy = dy_fii(df=fii_papel)

# Plotando o dividend yield médio, máximo e mínimo
fig = make_subplots(
    rows=3, 
    cols=1,
    subplot_titles=[
        'Dividend Yield Médio',
        'Dividend Yield Máximo',
        'Dividend Yield Mínimo'
    ],
    vertical_spacing=0.1
    )

fig.add_trace(go.Bar(
    x=df_papel_dy.index,
    y=df_papel_dy['dy_medio'],
    name='DY médio'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=df_papel_dy.index,
    y=df_papel_dy['dy_max'],
     name='DY máximo'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=df_papel_dy.index,
    y=df_papel_dy['dy_min'],
     name='DY mínimo'
), row=3, col=1)

fig.update_layout(
    title='Dividend yield dos FII de Papel',
    height=1200)

fig.show()

In [ ]:
# DY acumulado dos últimos 12 meses
df_papel_dy_accum = dy_fii_acumm_12m(df=fii_papel)

# Plotando o DY acumulados dos últimos 12 meses
fig = go.Figure()

for empresa in df_papel_dy_accum.columns:
    fig.add_trace(go.Scatter(
        x=df_papel_dy_accum.index,
        y=df_papel_dy_accum[empresa],
        name=empresa
    ))

fig.update_layout(title='Dividend yield acumulados dos últmos 12 meses - FII de Papel')

fig.show()

In [ ]:
# Retorno anual
ret_anual_papel = ret_anual(ticker=lst_ticker_fii_papel, setor='Papel', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_anual_papel = ret_anual_papel[0]

# Plot do retorno anual
ret_anual_papel[1];

In [ ]:
# Retorno acumulado
ret_acumulado_papel = ret_acumulado(ticker=lst_ticker_fii_papel, setor='Papel', df_benchmark=ifix_precos)

# Df para transformar em excel
df_ret_acumulado_papel = ret_acumulado_papel[0]

# Plot do retorno anual
ret_acumulado_papel[1];

In [ ]:
# Relação risco-retorno
plot_risk_return(ticker=lst_ticker_fii_papel, setor='Papel')

#### Análisando a carteira de CRIs dos FIIs

In [ ]:
# KNIP11
df_cris_knip = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//cris//knip11_062025.xlsx',
    sheet_name='Carteira de Ativos',
    header=10
)

# Removendo colunas
df_cris_knip = df_cris_knip.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'AGENTE FIDUCIÁRIO'])

# Selecionando as principais colunas
df_cris_knip = df_cris_knip[[
    'ATIVO', 
    'DEVEDOR / ATIVO IMOBILIARIO', 
    'INDEX.', 
    '% DA CARTEIRA', 
    'TAXA AQUISIÇÃO', 
    'TAXA MTM', 
    'LTV', 
    'VENCIMENTO DO CRI'
]]

# Arrumando o dtypes das colunas
df_cris_knip['TAXA AQUISIÇÃO'] = df_cris_knip['TAXA AQUISIÇÃO'].replace('-', 0).astype(float)
df_cris_knip['TAXA MTM'] = df_cris_knip['TAXA MTM'].replace('-', 0).astype(float)

# Preenchendo os NaN
df_cris_knip['LTV'] = df_cris_knip['LTV'].fillna(0)

# Análise do LTV
ltv_knip_media_ponderada = round(np.average(df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI', 'LTV'], weights=df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI', '% DA CARTEIRA']) * 100, 2)
ltv_knip_media = round(df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI', 'LTV'].mean() * 100, 2)
ltv_knip_max = round(df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI', 'LTV'].max() * 100, 2)
ltv_knip_min = round(df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI', 'LTV'].min() * 100, 2)
ltv_knip_num_zero = len(df_cris_knip.loc[(df_cris_knip['ATIVO'] == 'CRI') & (df_cris_knip['LTV'] == 0)])

# Números total de ativos
num_ativos_knip = len(df_cris_knip)

# Número total de CRIs
num_cris_knip = len(df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI'])

# Tipos de ativos no portfólio
ativos_knip = df_cris_knip['ATIVO'].unique()
# Transformando a lista em uma string única
ativos_knip = ', '.join(ativos_knip) 

# Contando os números de cada ativo do KNIP11
tipo_ativos_knip = df_cris_knip.loc[:, 'ATIVO'].value_counts()

# Contagem dos CRIs no KNIP11 por índice
tipo_cri_knip = df_cris_knip.loc[df_cris_knip['ATIVO'] == 'CRI', 'INDEX.'].value_counts()

# Tipos de indexadores
indexadores_knip = tipo_cri_knip.index
# Transformando a lista em uma string única
indexadores_knip = ', '.join(indexadores_knip)
# Retirando a string '+'
indexadores_knip = indexadores_knip.replace('+', '').strip()

# Plotando o subplot
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[[{'type':'domain'}], [{'type':'domain'}]],
    subplot_titles=(
        'Distribuição dos ativos',
        'Distribuição dos indexadores dos CRIs'),
    vertical_spacing=0.08  # Aumentando o espaçamento vertical
)

fig.add_trace(go.Pie(
    values=tipo_ativos_knip, 
    labels=tipo_ativos_knip.index,
    legendgroup='ativos_knip',  
    showlegend=True,
    textposition='inside',
    ), 
    row=1, col=1
)

fig.add_trace(go.Pie(
    values=tipo_cri_knip, 
    labels=tipo_cri_knip.index,
    legendgroup='cri_knip', 
    showlegend=True,
    textposition='inside',
    ), 
    row=2, col=1
)

fig.update_layout(
    title_text='Distribuição dos Ativos e Indexadores dos CRIs - KNIP11',
    height=1000
)

fig.show()

In [ ]:
# CRIs que estão ganhando com marcação a mercado
df_cris_knip.loc[df_cris_knip['TAXA AQUISIÇÃO'] > df_cris_knip['TAXA MTM']]

In [ ]:
# KNCR11
df_cris_kncr = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//cris//kncr11_062025.xlsx',
    sheet_name='Carteira de Ativos',
    header=10
)

# Removendo colunas
df_cris_kncr = df_cris_kncr.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'AGENTE FIDUCIÁRIO'])

# Selecionando as principais colunas
df_cris_kncr = df_cris_kncr[[
    'ATIVO', 
    'DEVEDOR / ATIVO IMOBILIARIO', 
    'INDEX.', 
    '% DA CARTEIRA', 
    'TAXA AQUISIÇÃO', 
    'TAXA MTM', 
    'LTV', 
    'VENCIMENTO DO CRI'
]]

# Arrumando o dtypes das colunas
df_cris_kncr['TAXA AQUISIÇÃO'] = df_cris_kncr['TAXA AQUISIÇÃO'].replace('-', 0).astype(float)
df_cris_kncr['TAXA MTM'] = df_cris_kncr['TAXA MTM'].replace('-', 0).astype(float)

# Preenchendo os NaN
df_cris_kncr['LTV'] = df_cris_kncr['LTV'].fillna(0)

# Análise do LTV
ltv_kncr_media_ponderada = round(np.average(df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI', 'LTV'], weights=df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI', '% DA CARTEIRA']) * 100, 2)
ltv_kncr_media = round(df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI', 'LTV'].mean() * 100, 2)
ltv_kncr_max = round(df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI', 'LTV'].max() * 100, 2)
ltv_kncr_min = round(df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI', 'LTV'].min() * 100, 2)
ltv_kncr_num_zero = len(df_cris_kncr.loc[(df_cris_kncr['ATIVO'] == 'CRI') & (df_cris_kncr['LTV'] == 0)])

# Números total de ativos
num_ativos_kncr = len(df_cris_kncr)

# Número total de CRIs
num_cris_kncr = len(df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI'])

# Tipos de ativos no portfólio
ativos_kncr = df_cris_kncr['ATIVO'].unique()
# Transformando a lista em uma string única
ativos_kncr = ', '.join(ativos_kncr) 

# Contando os números de cada ativo do KNCR11
tipo_ativos_kncr = df_cris_kncr.loc[:, 'ATIVO'].value_counts()

# Contagem dos CRIs no KNCR11 por índice
tipo_cri_kncr = df_cris_kncr.loc[df_cris_kncr['ATIVO'] == 'CRI', 'INDEX.'].value_counts()

# Tipos de indexadores
indexadores_kncr = tipo_cri_kncr.index
# Transformando a lista em uma string única
indexadores_kncr = ', '.join(indexadores_kncr)
# Retirando a string '+'
indexadores_kncr = indexadores_kncr.replace('+', '').strip()

# Plotando o subplot
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[[{'type':'domain'}], [{'type':'domain'}]],
    subplot_titles=(
        'Distribuição dos ativos',
        'Distribuição dos indexadores dos CRIs'),
    vertical_spacing=0.08  # Aumentando o espaçamento vertical
)

fig.add_trace(go.Pie(
    values=tipo_ativos_kncr, 
    labels=tipo_ativos_kncr.index,
    legendgroup='ativos_kncr',  
    showlegend=True,
    textposition='inside',
    ), 
    row=1, col=1
)

fig.add_trace(go.Pie(
    values=tipo_cri_kncr, 
    labels=tipo_cri_kncr.index,
    legendgroup='cri_kncr', 
    showlegend=True,
    textposition='inside',
    ), 
    row=2, col=1
)

fig.update_layout(
    title_text='Distribuição dos Ativos e Indexadores dos CRIs - KNCR11',
    height=1000
)

fig.show()

In [ ]:
# CRIs que estão ganhando com marcação a mercado
df_cris_kncr.loc[df_cris_kncr['TAXA AQUISIÇÃO'] > df_cris_kncr['TAXA MTM']]

In [ ]:
# KNSC11
df_cris_knsc = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//cris//knsc11_052025.xlsx',
    sheet_name='Carteira de Ativos',
    header=10
)

# Removendo colunas
df_cris_knsc = df_cris_knsc.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'AGENTE FIDUCIÁRIO'])

# Selecionando as principais colunas
df_cris_knsc = df_cris_knsc[[
    'ATIVO', 
    'DEVEDOR / ATIVO IMOBILIARIO', 
    'INDEX.', 
    '% DA CARTEIRA', 
    'TAXA AQUISIÇÃO', 
    'TAXA MTM', 
    'LTV', 
    'VENCIMENTO DO CRI'
]]

# Arrumando o dtypes das colunas
df_cris_knsc['TAXA AQUISIÇÃO'] = df_cris_knsc['TAXA AQUISIÇÃO'].replace('-', 0).astype(float)
df_cris_knsc['TAXA MTM'] = df_cris_knsc['TAXA MTM'].replace('-', 0).astype(float)

# Preenchendo os NaN
df_cris_knsc['LTV'] = df_cris_knsc['LTV'].fillna(0)

# Análise do LTV
ltv_knsc_media_ponderada = round(np.average(df_cris_knsc.loc[df_cris_knsc['ATIVO']=='CRI', 'LTV'], weights=df_cris_knsc.loc[df_cris_knsc['ATIVO']=='CRI', '% DA CARTEIRA']) * 100, 2)
ltv_knsc_media = round(df_cris_knsc.loc[df_cris_knsc['ATIVO']=='CRI', 'LTV'].mean() * 100, 2)
ltv_knsc_max = round(df_cris_knsc.loc[df_cris_knsc['ATIVO']=='CRI', 'LTV'].max() * 100, 2)
ltv_knsc_min = round(df_cris_knsc.loc[df_cris_knsc['ATIVO']=='CRI', 'LTV'].min() * 100, 2)
ltv_knsc_num_zero = len(df_cris_knsc.loc[(df_cris_knsc['ATIVO'] == 'CRI') & (df_cris_knsc['LTV'] == 0)])

# Números total de ativos
num_ativos_knsc = len(df_cris_knsc)

# Número total de CRIs
num_cris_knsc = len(df_cris_knsc.loc[df_cris_knsc['ATIVO'] == 'CRI'])

# Tipos de ativos no portfólio
ativos_knsc = df_cris_knsc['ATIVO'].unique()
# Transformando a lista em uma string única
ativos_knsc = ', '.join(ativos_knsc) 

# Contando os números de cada ativo do KNCR11
tipo_ativos_knsc = df_cris_knsc.loc[:, 'ATIVO'].value_counts()

# Contagem dos CRIs no KNCR11 por índice
tipo_cri_knsc = df_cris_knsc.loc[df_cris_knsc['ATIVO'] == 'CRI', 'INDEX.'].value_counts()

# Tipos de indexadores
indexadores_knsc = tipo_cri_knsc.index
# Transformando a lista em uma string única
indexadores_knsc = ', '.join(indexadores_knsc)
# Retirando a string '+'
indexadores_knsc = indexadores_knsc.replace('+', '').strip()

# Plotando o subplot
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[[{'type':'domain'}], [{'type':'domain'}]],
    subplot_titles=(
        'Distribuição dos ativos',
        'Distribuição dos indexadores dos CRIs'),
    vertical_spacing=0.08  # Aumentando o espaçamento vertical
)

fig.add_trace(go.Pie(
    values=tipo_ativos_knsc, 
    labels=tipo_ativos_knsc.index,
    legendgroup='ativos_knsc',  
    showlegend=True,
    textposition='inside',
    ), 
    row=1, col=1
)

fig.add_trace(go.Pie(
    values=tipo_cri_knsc, 
    labels=tipo_cri_knsc.index,
    legendgroup='cri_knsc', 
    showlegend=True,
    textposition='inside',
    ), 
    row=2, col=1
)

fig.update_layout(
    title_text='Distribuição dos Ativos e Indexadores dos CRIs - KNSC11',
    height=1000
)

fig.show()

In [ ]:
# CRIs que estão ganhando com marcação a mercado
df_cris_knsc.loc[df_cris_knsc['TAXA AQUISIÇÃO'] > df_cris_knsc['TAXA MTM']]

In [ ]:
# A partir de 2025, ele passou a disponibilizar apenas um arquivo XML e não mais Excel

# HGCR11
df_cris_hgcr = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//cris//hgcr11_062025.xlsx',
    sheet_name='Carteira',
    header=5,
    skipfooter=1
)

# Removendo as colunas
df_cris_hgcr = df_cris_hgcr.drop(columns=['Unnamed: 0', '#'])

# Formantando as colunas
df_cris_hgcr.columns = df_cris_hgcr.columns.str.replace('\n', '')
df_cris_hgcr = df_cris_hgcr.replace('-', 0)
df_cris_hgcr['LTV'] = df_cris_hgcr['LTV'].astype(float)

# Análise do LTV
ltv_hgcr_media_ponderada = round(np.average(df_cris_hgcr['LTV'], weights=df_cris_hgcr['%/PL']) * 100, 2)
ltv_hgcr_media = round(df_cris_hgcr['LTV'].mean() * 100, 2)
ltv_hgcr_max = round(df_cris_hgcr['LTV'].max() * 100, 2)
ltv_hgcr_min = round(df_cris_hgcr['LTV'].min() * 100, 2)
ltv_hgcr_num_zero = len(df_cris_hgcr.loc[df_cris_hgcr['LTV'] == 0])

# Números total de ativos
num_ativos_hgcr = len(df_cris_hgcr)

# Contagem dos CRIs no HGCR11 por índice
tipo_cri_hgcr = df_cris_hgcr.loc[:, 'Indexador'].value_counts()

# Removendo o index 0 - o fundo começou a comprar outros FIIs (não possuem indexadores)
tipo_cri_hgcr = tipo_cri_hgcr.drop(0)

# Tipos de indexadores
indexadores_hgcr = tipo_cri_hgcr.index

# Transformando a lista em uma string única
indexadores_hgcr = ', '.join(indexadores_hgcr)

# Criando o gráfico de pizza com plotly
fig = px.pie(
    values=tipo_cri_hgcr, 
    names=tipo_cri_hgcr.index, 
    title='Distribuição dos Indexadores dos CRIs - HGCR11'
)

fig.show()

* No arquivo do MXRF11, na aba 'Portfolio', existem 4 tabelas diferentes:
    * 'Carteira de CRIs';
    * 'Carteira de FIIs';
    * 'Carteira de Permutas Financeiras';
    * 'Imóveis destinados à venda'.

* **CONFERIR o range das linhas que contém a 'Carteira de CRIs'.**

In [ ]:
# MXRF11
df_cris_mxrf = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//cris//mxrf11_062025.xlsx',
    sheet_name='Portfolio',
    header=4
)

# Removendo a coluna
df_cris_mxrf = df_cris_mxrf.drop(columns=['Unnamed: 0'])

# CONFERIR o range das linhas (index) que contém a 'Carteira de CRIs'
df_cris_mxrf = df_cris_mxrf.iloc[0:98]

# Selecionando as principais colunas
df_cris_mxrf = df_cris_mxrf[[
    'Devedor',
    'Index.',
    '% do PL',
    'Taxa Emissão',
    'Vencimento'
]]

# Números total de ativos
num_ativos_mxrf = len(df_cris_mxrf)

# Contagem dos CRIs no MXRF11 por índice
tipo_cri_mxrf = df_cris_mxrf.loc[:, 'Index.'].value_counts()

# Tipos de indexadores
indexadores_mxrf = tipo_cri_mxrf.index
# Transformando a lista em uma string única
indexadores_mxrf = ', '.join(indexadores_mxrf)
# Retirando a string '+'
indexadores_mxrf = indexadores_mxrf.replace('+', '').strip()

# Criando o gráfico de pizza com plotly
fig = px.pie(
    values=tipo_cri_mxrf, 
    names=tipo_cri_mxrf.index, 
    title='Distribuição dos Indexadores dos CRIs - MXRF11'
)

fig.show()

In [ ]:
# VGIR11
df_cris_vgir = pd.read_excel(
    io='C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//cris//vgir11_062025.xlsx',
    header=4,
    skipfooter=7
)

# Selecionando as principais colunas
df_cris_vgir = df_cris_vgir[['Ativo', 'Indexador', '% PL', 'Cupom', 'LTV', 'Vencimento']]

# Arrumando o dtypes das colunas
df_cris_vgir['LTV'] = df_cris_vgir['LTV'].replace('na', 0).astype(float)

# Análise do LTV
ltv_vgir_media_ponderada = round(np.average(df_cris_vgir['LTV'], weights=df_cris_vgir['% PL']) * 100, 2)
ltv_vgir_media = round(df_cris_vgir['LTV'].mean() * 100, 2)
ltv_vgir_max = round(df_cris_vgir['LTV'].max() * 100, 2)
ltv_vgir_min = round(df_cris_vgir['LTV'].min() * 100, 2)
ltv_vgir_num_zero = len(df_cris_vgir.loc[df_cris_vgir['LTV'] == 0])

# Números total de ativos
num_ativos_vgir = len(df_cris_vgir)

# Contagem dos CRIs no VGIR11 por índice
tipo_cri_vgir = df_cris_vgir.loc[:, 'Indexador'].value_counts()

# Tipos de indexadores
indexadores_vgir = tipo_cri_vgir.index
# Transformando a lista em uma string única
indexadores_vgir = ', '.join(indexadores_vgir)
# Retirando a string '+'
indexadores_vgir = indexadores_vgir.replace('+', '').strip()

# Criando o gráfico de pizza com plotly
fig = px.pie(
    values=tipo_cri_vgir, 
    names=tipo_cri_vgir.index, 
    title='Distribuição dos Indexadores dos CRIs - VGIR11'
)

fig.show()

In [ ]:
# Dicionário com as informações do portfólio dos FIIs de papeis
dict_portfolio_papel = {
    'num_total_ativos': [
        num_ativos_knip,
        num_ativos_kncr, 
        num_ativos_knsc,
        num_ativos_hgcr,
        num_ativos_mxrf,
        num_ativos_vgir
    ],
    # No arquivo excel, os fundos das Kinea mostram as outras compras de ativos como LCI, FIIs e Caixa
    'tipos_ativos': [
        ativos_knip,
        ativos_kncr, 
        ativos_knsc,
        'CRI',
        'CRI',
        'CRI'
    ],
    'num_cris': [
        num_cris_knip, 
        num_cris_kncr, 
        num_cris_knsc,
        num_ativos_hgcr,
        num_ativos_mxrf,
        num_ativos_vgir
    ],
    'indexador_principal':[
        tipo_cri_knip.index[0],
        tipo_cri_kncr.index[0],
        tipo_cri_knsc.index[0],
        tipo_cri_hgcr.index[0],
        tipo_cri_mxrf.index[0],
        tipo_cri_vgir.index[0]
    ],
    'tipo_indexadores':[
        indexadores_knip,
        indexadores_kncr,
        indexadores_knsc,
        indexadores_hgcr,
        indexadores_mxrf,
        indexadores_vgir
    ],
    'ltv_media_ponderada': [
        ltv_knip_media_ponderada,
        ltv_kncr_media_ponderada,
        ltv_knsc_media_ponderada,
        ltv_hgcr_media_ponderada,
        52, # MXRF11 divulga o LTV médio no arquivo 'Relatório Gerencial'
        ltv_vgir_media_ponderada
    ],
    'ltv_media': [
        ltv_knip_media,
        ltv_kncr_media,
        ltv_knsc_media,
        ltv_hgcr_media,
        52, # MXRF11 divulga o LTV médio no arquivo 'Relatório Gerencial'
        ltv_vgir_media
    ],
    'ltv_max': [
        ltv_knip_max,
        ltv_kncr_max,
        ltv_knsc_max,
        ltv_hgcr_max,
        0, # MXRF11 não divulga o ltv no arquivo excel
        ltv_vgir_max
    ],
    'ltv_min': [
        ltv_knip_min,
        ltv_kncr_min,
        ltv_knsc_min,
        ltv_hgcr_min,
        0, # MXRF11 não divulga o ltv no arquivo excel
        ltv_vgir_min
    ],
    'num_ltv_zero': [
        ltv_knip_num_zero,
        ltv_kncr_num_zero,
        ltv_knsc_num_zero,
        ltv_hgcr_num_zero,
        0, # MXRF11 não divulga o ltv no arquivo excel
        ltv_vgir_num_zero
    ]
}

# Transformando em um df
df_portfolio_papel = pd.DataFrame(dict_portfolio_papel, index=lst_ticker_fii_papel)

# Retirando a string '+' da coluna 'indexador_principal'
df_portfolio_papel['indexador_principal'] = df_portfolio_papel['indexador_principal'].str.replace('+','', regex=True).str.strip()
df_portfolio_papel

In [ ]:
# Transformando em um arquivo excel
with pd.ExcelWriter(f'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//fii//fii_papel.xlsx') as writer:
    fii_knip.to_excel(writer, sheet_name='KNIP11')
    fii_kncr.to_excel(writer, sheet_name='KNCR11')
    fii_knsc.to_excel(writer, sheet_name='KNSC11')
    fii_hgcr.to_excel(writer, sheet_name='HGCR11')
    fii_mxrf.to_excel(writer, sheet_name='MXRF11')
    fii_vgir.to_excel(writer, sheet_name='VGIR11')
    df_papel_dy.to_excel(writer, sheet_name='DIVIDEND_YIELD')
    df_papel_dy_accum.to_excel(writer, sheet_name='DIVIDEND_YIELD_ACUM')
    df_vol_drawdown_papel.to_excel(writer, sheet_name='VOLATILIDADE_DRAWDOWN')
    df_ret_anual_papel.to_excel(writer, sheet_name='RET_ANUAL')
    df_ret_acumulado_papel.to_excel(writer, sheet_name='RET_ACUM')
    df_portfolio_papel.to_excel(writer, sheet_name='PORTFOLIO_CRIS')

#### Comparando a taxa de retorno (MTM) dos FIIs com a taxa dos títulos públicos

* Vídeo: https://www.youtube.com/watch?v=rMaNf6ycss0
* Última atualização feita em 16/08/2024:
    * Os dados do MTM (Taxa de Retorno) dos fundos estão no "Relatório Gerencial" (divulgado mensalmente);
    * Download do arquivo csv "PrecoTaxaTesouroDireto" ("Histórico Geral" - "Consolidado Preços e Taxas") -> https://www.tesourodireto.com.br/titulos/historico-de-precos-e-taxas.htm

In [ ]:
# Meta Selic (432)
meta_selic = consulta_bc(codigo_bcb='432', data_inicial='01/01/2024', data_final='01/04/2025')
# Selecionando a última meta selic
ultima_meta_selic = meta_selic['valor'][-1]

# CDI (4392) - Taxa de juros - CDI acumulada no mês anualizada base 252	
cdi_anualizado = consulta_bc(codigo_bcb='4392', data_inicial='01/01/2024', data_final='01/04/2025')
# Selecionando o último cdi anualizado
ultimo_cdi_anualizado = cdi_anualizado['valor'][-1]

# IPCA (433) - Índice nacional de preços ao consumidor-amplo (mensal)
ipca =  consulta_bc(codigo_bcb='433', data_inicial='01/01/2024', data_final='01/04/2025')
# Calculando o IPCA acumulado dos últimos 12 meses
ipca['taxa_unit'] = (ipca / 100) + 1
ipca['ipca_acumulado'] = round((ipca['taxa_unit'].rolling(window=12).agg(lambda x : x.prod()) -1) * 100, 2)
# Selecionando o último IPCA acumulado dos últimos 12 meses
ultimo_ipca_acumulado = ipca['ipca_acumulado'][-1]

* **Atualizar os números do 'MTM_IPCA' e 'MTM_CDI' do dicionário 'dict_fii_papel'**

In [ ]:
# Lista dos tickers - o VGIR não mostra qual é o seu MTM
lst_ticker_fii_papel = [
    'KNIP11.SA', 
    'KNCR11.SA', 
    'KNSC11.SA', 
    'HGCR11.SA', 
    'MXRF11.SA'
]

dict_fii_papel = {
    'Indexador': [
        'IPCA',   # KNIP11
        'CDI',    # KNCR11
        'MISTO',  # KNSC11
        'MISTO',  # HGCR11
        'MISTO'   # MXRF11
    ], 
    'MTM_IPCA':[
        10.59,    # KNIP11
        0,        # KNCR11
        10.65,    # KNSC11
        10.40,    # HGCR11
        8.63      # MXRF11
    ],
    'MTM_CDI': [
        0,        # KNIP11
        2.24,     # KNCR11
        3.31,     # KNSC11
        4.10,     # HGCR11 
        2.74      # MXRF11
    ],
}

# Transformando em um df
df_fii_papel = pd.DataFrame(dict_fii_papel, index=lst_ticker_fii_papel)

# Soma do 'MTM_IPCA' com o IPCA dos últimos 12 meses
df_fii_papel['Taxa_IPCA'] = df_fii_papel['MTM_IPCA'] + ultimo_ipca_acumulado
df_fii_papel.loc['KNCR11.SA', 'Taxa_IPCA'] = 0

# Soma do 'MTM_CDI' com o CDI anualizado
df_fii_papel['Taxa_CDI'] = df_fii_papel['MTM_CDI'] + ultimo_cdi_anualizado
df_fii_papel.loc['KNIP11.SA', 'Taxa_CDI'] = 0

In [ ]:
# Lendo o arquivo
df_titulo_publico = pd.read_csv(
    'C://Users//vitor//projetos_python//python_b3//composicao-fundos-de-investimentos//fundo_imobiliario//titulos_publicos//PrecoTaxaTesouroDireto.csv', 
    sep=';'
)

# Transformando as colunas em datetime
df_titulo_publico['Data Vencimento'] = pd.to_datetime(df_titulo_publico['Data Vencimento'], format='%d/%m/%Y')
df_titulo_publico['Data Base'] = pd.to_datetime(df_titulo_publico['Data Base'], format='%d/%m/%Y')
# Trocando os ',' por '.'
df_titulo_publico['Taxa Compra Manha'] = df_titulo_publico['Taxa Compra Manha'].str.replace(',', '.').astype(float)
df_titulo_publico['Taxa Venda Manha'] = df_titulo_publico['Taxa Venda Manha'].str.replace(',', '.').astype(float)
df_titulo_publico['PU Compra Manha'] = df_titulo_publico['PU Compra Manha'].str.replace(',', '.').astype(float)
df_titulo_publico['PU Venda Manha'] = df_titulo_publico['PU Venda Manha'].str.replace(',', '.').astype(float)
df_titulo_publico['PU Base Manha'] = df_titulo_publico['PU Base Manha'].str.replace(',', '.').astype(float)

# Tesouro IPCA+ 2035
filt = (df_titulo_publico['Tipo Titulo'] == 'Tesouro IPCA+') & (df_titulo_publico['Data Vencimento'] == '2035-05-15')
tesouro_ipca_2035 = df_titulo_publico.loc[filt].sort_values('Data Base')
# Selecionando a última taxa do IPCA+ 2035
ultima_taxa_tesouro_ipca_2035 = tesouro_ipca_2035['Taxa Compra Manha'].iloc[-1]

# Tesouro SELIC 2029
filt = (df_titulo_publico['Tipo Titulo'] == 'Tesouro Selic') & (df_titulo_publico['Data Vencimento'] == '2029-03-01')
tesouro_selic_2029 = df_titulo_publico.loc[filt].sort_values('Data Base')
# Selecionando a última taxa do SELIC 2029
ultima_taxa_tesouro_selic_2029 = tesouro_selic_2029['Taxa Compra Manha'].iloc[-1]

# Títulos públicos 
lst_titulo_publico = ['IPCA_2035', 'SELIC_2029']

dict_titulo_publico = {
    'Ultima_Taxa': [ultima_taxa_tesouro_ipca_2035, ultima_taxa_tesouro_selic_2029],
    'Taxa_Indexador': [ultimo_ipca_acumulado, ultima_meta_selic]
}

# Transformando em um df
df_titulo_publico_final = pd.DataFrame(dict_titulo_publico, index=lst_titulo_publico)

# Calculando a taxa dos títulos públicos
df_titulo_publico_final['Taxa_Titulo'] = df_titulo_publico_final['Ultima_Taxa'] + df_titulo_publico_final['Taxa_Indexador']

In [ ]:
print('='*60)
print(f'A meta SELIC é de {ultima_meta_selic}')
print(f'O IPCA de 12 meses é de {ultimo_ipca_acumulado}')
print(f'O CDI anualizado é de {ultimo_cdi_anualizado}')
print('='*60)
print(df_fii_papel)
print('='*60)
print(df_titulo_publico_final)
print('='*60)

In [ ]:
# Calculando a diferença entre a taxa do FII e a taxa do título
df_fii_papel['Diferenca_IPCA'] = df_fii_papel['Taxa_IPCA'] - df_titulo_publico_final.loc['IPCA_2035' ,'Taxa_Titulo']
df_fii_papel.loc['KNCR11.SA', 'Diferenca_IPCA'] = 0

df_fii_papel['Diferenca_CDI'] = df_fii_papel['Taxa_CDI'] - df_titulo_publico_final.loc['SELIC_2029' ,'Taxa_Titulo']
df_fii_papel.loc['KNIP11.SA', 'Diferenca_CDI'] = 0

# Ordenando essa diferença
df_fii_papel_diff_ipca = df_fii_papel['Diferenca_IPCA'].sort_values()
df_fii_papel_diff_cdi = df_fii_papel['Diferenca_CDI'].sort_values()

# Plotando o heatmap
fig, axes = plt.subplots(ncols=2, figsize=(12, 6))
fig.suptitle('Diferença entre a taxa do FII e a taxa do título público', fontsize=16)

sns.heatmap(
    df_fii_papel_diff_ipca.values.reshape(-1, 1), 
    annot=True, # Plotando os números dentro do gráfico
    fmt=".2f",  # Formatação para 2 casas decimais
    cmap='coolwarm', 
    yticklabels=df_fii_papel_diff_ipca.index,
    ax=axes[0]
)
axes[0].set_xticks([0.5])
axes[0].set_xticklabels(['Diferenca_IPCA'], rotation=0)

sns.heatmap(
    df_fii_papel_diff_cdi.values.reshape(-1, 1), 
    annot=True, # Plotando os números dentro do gráfico
    fmt=".2f",  # Formatação para 2 casas decimais
    cmap='coolwarm', 
    yticklabels=df_fii_papel_diff_cdi.index,
    ax=axes[1]
)
axes[1].set_xticks([0.5])
axes[1].set_xticklabels(['Diferenca_CDI'], rotation=0)

plt.show()

**Interpretação**
* Quanto maior for a diferença entre a MTM e a taxa dos títulos públicos, o FII é mais arriscado (High yield - compra CRIs que possuem maiores riscos);
* Quanto menor for a diferença entre a MTM e a taxa dos títulos públicos, o FII é mais seguro (High grade - compra CRIs que possuem menores riscos).
